In [5]:
######################### Stage 0 — Setup and Configuration  ########################
# 
# Define paths once and verify raw inputs exist before we compute anything.
# HOW
#   - Choose APP_ROOT automatically (env var > common path > current working dir).
#   - Define subfolders: data/, configs/, final_results/ (create final_results/ if missing).
#   - For each sample under data/: check for expression matrices and spatial files.
# WHAT IT PRODUCES
#   - Variables: APP_ROOT, DATA, CONFIGS, FINAL
#   - Folder: final_results/ (created if missing)
#   - Console table of samples and file completeness (no hidden loading).
# INTERPRETATION
#   - “ready_for_run=True” means the sample has counts + spatial positions; it can be used immediately.
#   - If a sample is missing matrices, drop its raw files into data/<sample>/ and re-run this cell.

import os, time
from pathlib import Path
import pandas as pd

def now(): return time.strftime("%H:%M:%S")

# --------- pick APP_ROOT (no prompts) ---------
candidates = []
if os.environ.get("IMMUNE_APP_ROOT"):
    candidates.append(Path(os.environ["IMMUNE_APP_ROOT"]).expanduser())
# your known local path (adjust if different)
candidates.append(Path("/Users/sally/Desktop/Immune_Exclusion_Infiltration").expanduser())
# fallback: current working directory
candidates.append(Path.cwd())

APP_ROOT = next((p for p in candidates if p.exists()), candidates[-1]).resolve()
DATA     = APP_ROOT / "data"
CONFIGS  = APP_ROOT / "configs"
FINAL    = APP_ROOT / "final_results"
FINAL.mkdir(parents=True, exist_ok=True)

print(f"[{now()}] APP_ROOT: {APP_ROOT}")
print(f"[{now()}] data/ exists    : {DATA.exists()}")
print(f"[{now()}] configs/ exists : {CONFIGS.exists()}")
print(f"[{now()}] final_results/  : {FINAL.exists()}  (created if missing)")

# --------- audit helpers ---------
def has_any(p: Path, patterns):
    for pat in patterns:
        if list(p.glob(pat)):
            return True
    return False

def audit_sample(sample_dir: Path):
    # expression
    tenx_h5   = has_any(sample_dir, ["**/filtered_feature_bc_matrix.h5"])
    mtx_trip  = (has_any(sample_dir, ["matrix.mtx", "matrix.mtx.gz", "*/matrix.mtx", "*/matrix.mtx.gz"]) and
                 has_any(sample_dir, ["barcodes.tsv", "barcodes.tsv.gz", "*/barcodes.tsv", "*/barcodes.tsv.gz"]) and
                 has_any(sample_dir, ["features.tsv", "features.tsv.gz", "genes.tsv", "genes.tsv.gz",
                                      "*/features.tsv", "*/features.tsv.gz", "*/genes.tsv", "*/genes.tsv.gz"]))
    h5ad_file = has_any(sample_dir, ["*.h5ad"])
    # spatial
    scalef    = has_any(sample_dir, ["**/spatial/scalefactors_json.json"])
    hires     = has_any(sample_dir, ["**/spatial/tissue_hires_image.png"])
    lowres    = has_any(sample_dir, ["**/spatial/tissue_lowres_image.png"])
    pos_file  = has_any(sample_dir, ["**/spatial/tissue_positions*.csv",
                                     "**/spatial/tissue_positions*.tsv",
                                     "**/spatial/tissue_positions.parquet"])
    # ready when we have expression (any format) + positions
    has_expr  = tenx_h5 or mtx_trip or h5ad_file
    ready     = bool(has_expr and pos_file)
    return {
        "sample_id": sample_dir.name,
        "path": str(sample_dir),
        "10x_h5": tenx_h5,
        "mtx_triplet": mtx_trip,
        "h5ad": h5ad_file,
        "scalefactors_json": scalef,
        "tissue_hires_png": hires,
        "tissue_lowres_png": lowres,
        "tissue_positions": pos_file,
        "ready_for_run": ready,
    }

# --------- audit data/ ---------
rows = []
if DATA.exists():
    for p in sorted([d for d in DATA.iterdir() if d.is_dir()]):
        rows.append(audit_sample(p))

audit_df = pd.DataFrame(rows) if rows else pd.DataFrame(columns=[
    "sample_id","path","10x_h5","mtx_triplet","h5ad","scalefactors_json","tissue_hires_png","tissue_lowres_png","tissue_positions","ready_for_run"
])

# pretty print (no display(); VS Code and Jupyter both show prints)
print("\n[INFO] Detected {} sample(s) under data/:".format(len(audit_df)))
if not audit_df.empty:
    # reorder columns for readability
    cols = ["sample_id","path","h5ad","10x_h5","mtx_triplet","scalefactors_json","tissue_hires_png","tissue_lowres_png","tissue_positions","ready_for_run"]
    cols = [c for c in cols if c in audit_df.columns]
    print(audit_df[cols].to_string(index=False, max_colwidth=48))
else:
    print("  (no sample folders found)")

# minimal env info (helps reproducibility, no heavy imports)
import sys
try:
    import numpy as _np; np_v = _np.__version__
except Exception:
    np_v = "n/a"
try:
    import pandas as _pd; pd_v = _pd.__version__
except Exception:
    pd_v = "n/a"

print(f"\n[ENV] python: {sys.version.split()[0]}")
print(f"[ENV] pandas: {pd_v} | numpy: {np_v}")

# Keep APP_ROOT/DATA/CONFIGS/FINAL in memory for the next cell.



[00:39:07] APP_ROOT: /Users/sally/Desktop/Immune_Exclusion_Infiltration
[00:39:07] data/ exists    : True
[00:39:07] configs/ exists : True
[00:39:07] final_results/  : True  (created if missing)

[INFO] Detected 9 sample(s) under data/:
                              sample_id                                             path  h5ad  10x_h5  mtx_triplet  scalefactors_json  tissue_hires_png  tissue_lowres_png  tissue_positions  ready_for_run
                              GSE190811 /Users/sally/Desktop/Immune_Exclusion_Infiltr... False    True        False               True              True              False              True           True
                              GSE217414 /Users/sally/Desktop/Immune_Exclusion_Infiltr... False    True        False               True              True               True              True           True
                              GSE226997 /Users/sally/Desktop/Immune_Exclusion_Infiltr... False    True        False               True             

In [6]:
######################### Stage 1 — Load and Preprocess Visium Data  ########################
# Loads representative slides from data/, attaches spatial coords, CPM→log1p, applies alias map,
# and collapses duplicate genes using a fast sparse method (not the slow pandas groupby).

# ---- settings ----
SELECTED_OVERRIDE = ['GSE274103', 'GSE226997', 'GSE217414']  # force these if present

import os, sys, time, json
from pathlib import Path
import numpy as np, pandas as pd

assert 'APP_ROOT' in globals() and 'DATA' in globals() and 'CONFIGS' in globals(), "Run Cell 0 first."

import scanpy as sc
import anndata as ad
from scipy import sparse as sp

def log(msg): print(msg, flush=True)
def first_match(folder: Path, patterns):
    for pat in patterns:
        hits = list(folder.glob(pat))
        if hits: return hits[0]
    return None

def load_alias_map(p: Path):
    if not p.exists(): return {}
    try:
        import yaml
        raw = yaml.safe_load(p.read_text()) or {}
        alias = {}
        for k, v in raw.items():
            if isinstance(v, (list, tuple)):
                can = str(k)
                for a in v: alias[str(a).upper()] = can
                alias[can.upper()] = can
            else:
                alias[str(k).upper()] = str(v); alias[str(v).upper()] = str(v)
        return alias
    except Exception as e:
        log(f"[alias] warn: {e}"); return {}

def apply_aliases(names, alias):
    if not alias: return pd.Index(names), 0.0
    out, hits = [], 0
    for g in names:
        key = str(g).upper()
        if key in alias: out.append(alias[key]); hits += 1
        else: out.append(str(g))
    return pd.Index(out), hits / max(1, len(names))

def read_positions(sample_dir: Path):
    s = sample_dir / "spatial"
    if not s.exists(): return None
    f = first_match(s, ["tissue_positions*.csv","tissue_positions*.tsv","tissue_positions.parquet"])
    if f is None: return None
    if f.suffix == ".parquet":
        df = pd.read_parquet(f)
    elif f.suffix == ".tsv":
        df = pd.read_csv(f, sep="\t", header=None)
    else:
        try:
            df = pd.read_csv(f)
            if df.shape[1] < 5: df = pd.read_csv(f, header=None)
        except Exception:
            df = pd.read_csv(f, header=None)
    if df.shape[1] >= 6:
        df = df.iloc[:, :6]; df.columns = ["barcode","in_tissue","row","col","y","x"]
    elif df.shape[1] == 5:
        df = df.iloc[:, :5]; df.columns = ["barcode","row","col","y","x"]; df["in_tissue"] = 1
    else:
        return None
    df["barcode"] = df["barcode"].astype(str)
    df["in_tissue"] = (df["in_tissue"] == 1) if df["in_tissue"].dtype != bool else df["in_tissue"]
    return df[["barcode","in_tissue","x","y"]]

def load_adata(sample_dir: Path):
    h5 = first_match(sample_dir, ["**/filtered_feature_bc_matrix.h5"])
    if h5:
        try: return sc.read_10x_h5(str(h5))
        except Exception as e: log(f"[warn] read_10x_h5 {sample_dir.name}: {e}")
    for d in [sample_dir, sample_dir/"filtered_feature_bc_matrix"]:
        mtx = first_match(d, ["matrix.mtx","matrix.mtx.gz"])
        bc  = first_match(d, ["barcodes.tsv","barcodes.tsv.gz"])
        ft  = first_match(d, ["features.tsv","features.tsv.gz","genes.tsv","genes.tsv.gz"])
        if mtx and bc and ft:
            try: return sc.read_10x_mtx(str(d), gex_only=False)
            except Exception as e: log(f"[warn] read_10x_mtx {d}: {e}")
    h5ad = first_match(sample_dir, ["*.h5ad"])
    if h5ad:
        try: return sc.read_h5ad(str(h5ad))
        except Exception as e: log(f"[warn] read_h5ad {sample_dir.name}: {e}")
    return None

def audit(sample_dir: Path):
    have_pos = bool(first_match(sample_dir, ["**/spatial/tissue_positions*.csv","**/spatial/tissue_positions*.tsv","**/spatial/tissue_positions.parquet"]))
    have_expr = bool(first_match(sample_dir, ["**/filtered_feature_bc_matrix.h5"]) or
                     first_match(sample_dir, ["*.h5ad"]) or
                     (first_match(sample_dir, ["matrix.mtx","matrix.mtx.gz"]) and
                      first_match(sample_dir, ["barcodes.tsv","barcodes.tsv.gz"]) and
                      first_match(sample_dir, ["features.tsv","features.tsv.gz","genes.tsv","genes.tsv.gz"])))
    score = (3 if first_match(sample_dir, ["**/filtered_feature_bc_matrix.h5"]) else 0) + (2 if have_expr else 0) + (2 if have_pos else 0)
    return {"sample_id": sample_dir.name, "dir": sample_dir, "have_pos": have_pos, "have_expr": have_expr, "score": score}

# --- discover & choose slides
cands = [d for d in DATA.iterdir() if d.is_dir()]
aud = pd.DataFrame([audit(p) for p in cands]).sort_values(["score","sample_id"], ascending=[False, True])
eligible = aud[aud["have_pos"] & aud["have_expr"]]
SELECTED = [s for s in SELECTED_OVERRIDE if (DATA/s).exists() and s in eligible["sample_id"].tolist()]
for s in eligible["sample_id"]:
    if len(SELECTED) == 3: break
    if s not in SELECTED: SELECTED.append(s)
log(f"[select] eligible={len(eligible)} | selected={SELECTED}")

# --- alias map
ALIAS_MAP = load_alias_map(CONFIGS / "gene_aliases.yaml")
log(f"[alias] entries={len(ALIAS_MAP)}")

# --- main: load + preprocess + fast duplicate collapse
PREPROC, rows = {}, []
for sid in SELECTED:
    log(f"[load] {sid} …")
    sd = DATA / sid
    adata = load_adata(sd)
    if adata is None:
        log(f"[skip] {sid}: no matrix"); rows.append({"sample_id":sid,"status":"no_matrix"}); continue
    if "gene_symbols" in adata.var.columns:
        adata.var_names = adata.var["gene_symbols"]

    pos = read_positions(sd)
    if pos is not None:
        idx = pd.Index(adata.obs_names.astype(str), name="barcode")
        j = pd.DataFrame(index=idx).join(pos.set_index("barcode"), how="left")
        adata.obs["in_tissue"] = j["in_tissue"].fillna(False).astype(bool)
        if {"x","y"}.issubset(j.columns):
            adata.obsm["spatial"] = j[["x","y"]].to_numpy(dtype=float)
    else:
        adata.obs["in_tissue"] = True
    adata = adata[adata.obs["in_tissue"]].copy()

    # CPM → log1p
    if sp.issparse(adata.X):
        lib = np.asarray(adata.X.sum(axis=1)).ravel(); lib[lib==0]=1.0
        X = adata.X.tocsr(); X = X.multiply(1e6 / lib[:,None]); X.data = np.log1p(X.data); adata.X = X
    else:
        lib = adata.X.sum(axis=1); lib[lib==0]=1.0; adata.X = np.log1p(adata.X * (1e6 / lib[:,None]))

    # alias + fast duplicate collapse
    new, cov = apply_aliases(adata.var_names, ALIAS_MAP)
    adata.var_names = new
    names = pd.Index(adata.var_names)
    if names.duplicated().any():
        X = adata.X.tocsr() if sp.issparse(adata.X) else sp.csr_matrix(adata.X)
        uniq, inv = np.unique(names.values, return_inverse=True)
        n_cols, n_groups = X.shape[1], uniq.size
        G = sp.csr_matrix((np.ones(n_cols, dtype=np.int8), (np.arange(n_cols), inv)), shape=(n_cols, n_groups))
        X_new = X @ G
        adata = ad.AnnData(X_new, obs=adata.obs.copy(), var=pd.DataFrame(index=uniq))
        if "spatial" in getattr(adata, "obsm", {}):
            adata.obsm["spatial"] = adata.obsm["spatial"]

    PREPROC[sid] = adata
    log(f"[ok]   {sid}: spots={adata.n_obs:,} genes={adata.n_vars:,} alias_cov={cov:.1%} spatial={'yes' if 'spatial' in adata.obsm_keys() else 'no'}")
    rows.append({"sample_id": sid, "status":"ok", "spots": int(adata.n_obs), "genes": int(adata.n_vars),
                 "alias_cov": float(cov), "spatial": bool("spatial" in adata.obsm_keys())})

# summary (visible + saved)
FINAL = Path(FINAL) if 'FINAL' in globals() else (APP_ROOT / "final_results"); FINAL.mkdir(parents=True, exist_ok=True)
summary = pd.DataFrame(rows)
summary.to_csv(FINAL / "preproc_summary.csv", index=False)
summary.sort_values("sample_id")


[select] eligible=5 | selected=['GSE274103', 'GSE226997', 'GSE217414']
[alias] entries=71
[load] GSE274103 …


/Users/sally/Desktop/Immune_Exclusion_Infiltration/.venv/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Users/sally/Desktop/Immune_Exclusion_Infiltration/.venv/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


[ok]   GSE274103: spots=4,952 genes=17,935 alias_cov=0.2% spatial=no
[load] GSE226997 …


/Users/sally/Desktop/Immune_Exclusion_Infiltration/.venv/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Users/sally/Desktop/Immune_Exclusion_Infiltration/.venv/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


[ok]   GSE226997: spots=4,148 genes=17,936 alias_cov=0.2% spatial=no
[load] GSE217414 …
[ok]   GSE217414: spots=2,887 genes=17,935 alias_cov=0.2% spatial=no


/Users/sally/Desktop/Immune_Exclusion_Infiltration/.venv/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Users/sally/Desktop/Immune_Exclusion_Infiltration/.venv/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


,sample_id,status,spots,genes,alias_cov,spatial
2,GSE217414,ok,2887,17935,0.001728,False
1,GSE226997,ok,4148,17936,0.001728,False
0,GSE274103,ok,4952,17935,0.001728,False


In [7]:
######################### Stage 2 — Build Spatial kNN Graph (tissue-only) ########################
# RATIONALE
#   Lock in the biology we’ll track (axes) and, if available, summarize roles per sample.
#   This cell is lightweight and always produces visible output.
#
# HOW
#   - If PREPROC[sid].obs['role'] exists, print simple counts (tumor/stroma/immune).
#   - Define the fixed IEvI axes (no file reads); keep them in AXES_DF for later steps.
#
# WHAT IT PRODUCES
#   - AXES_DF: DataFrame with ligand, receptor, axis, receptor_is_prefix.
#   - Console lines with role counts when present.
#   - The last expression returns AXES_DF so a table renders in the notebook.

import pandas as pd

# --- sanity on prerequisites ---
if 'PREPROC' not in globals() or not isinstance(PREPROC, dict) or len(PREPROC) == 0:
    print("[note] PREPROC is empty or missing. Re-run Cells 0–1. Proceeding to define axes anyway.")
    SAMPLES = []
else:
    SAMPLES = list(PREPROC.keys())
    print("[samples]", ", ".join(SAMPLES))

# --- summarize roles if already present in obs (optional) ---
for sid in SAMPLES:
    adata = PREPROC[sid]
    if "role" in adata.obs.columns:
        vc = adata.obs["role"].astype(str).str.lower().value_counts(dropna=False)
        t = int(vc.get("tumor", 0)); s = int(vc.get("stroma", 0)); i = int(vc.get("immune", 0))
        print(f"[roles] {sid}: tumor={t:,} | stroma={s:,} | immune={i:,} | total={adata.n_obs:,}")
    else:
        print(f"[roles] {sid}: (no obs['role'] column — OK)")

# --- fixed axes for IEvI (no file I/O) ---
AXES_DF = pd.DataFrame([
    {"ligand":"VEGFA",  "receptor":"KDR",   "axis":"VEGFA->KDR",      "receptor_is_prefix": False},
    {"ligand":"CXCL12","receptor":"CXCR4", "axis":"CXCL12->CXCR4",   "receptor_is_prefix": False},
    {"ligand":"TGFB1", "receptor":"TGFBR", "axis":"TGFB1->TGFBR*",   "receptor_is_prefix": True},   # any TGFBR*
    {"ligand":"SPP1",  "receptor":"ITG",   "axis":"SPP1->integrins", "receptor_is_prefix": True},   # any ITG*
])

# last expression -> renders a small table so you see output
AXES_DF


[samples] GSE274103, GSE226997, GSE217414
[roles] GSE274103: (no obs['role'] column — OK)
[roles] GSE226997: (no obs['role'] column — OK)
[roles] GSE217414: (no obs['role'] column — OK)


,ligand,receptor,axis,receptor_is_prefix
0,VEGFA,KDR,VEGFA->KDR,False
1,CXCL12,CXCR4,CXCL12->CXCR4,False
2,TGFB1,TGFBR,TGFB1->TGFBR*,True
3,SPP1,ITG,SPP1->integrins,True


In [8]:
######################### Stage 3 — Define Fixed Ligand–Receptor Axes  ########################
# RATIONALE
#   Our previous said "no spatial" because many datasets keep the Visium files in nested folders
#   (e.g., sample/subdir/spatial/...). Cell 1 read only sample/spatial, so coords weren’t attached.
#   This cell *repairs* PREPROC by finding positions files recursively, attaches coords, then builds kNN.
#
# HOW
#   - For each PREPROC sample, search recursively for `**/spatial/tissue_positions*.(csv|tsv|parquet)`.
#   - Parse with robust column handling (10x formats, header/no-header), set obsm["spatial"] = [x,y] in pixels.
#   - Keep/define obs["in_tissue"] (default True if missing).
#   - Build kNN (k=8) using scikit-learn if available, else a NumPy fallback.
#
# WHAT IT PRODUCES
#   - Updates PREPROC[sid].obsm["spatial"] in memory.
#   - GRAPH[sid] with kNN edges.
#   - Prints per-sample summaries and RETURNS a DataFrame (also saved to final_results/graph_summary.csv).

import numpy as np, pandas as pd, sys
from pathlib import Path

assert 'PREPROC' in globals() and isinstance(PREPROC, dict) and len(PREPROC) > 0, "Run Cells 0–1 first."
APP_ROOT = Path(APP_ROOT) if 'APP_ROOT' in globals() else Path.cwd()
DATA     = Path(DATA)     if 'DATA'     in globals() else (APP_ROOT/"data")
FINAL    = Path(FINAL)    if 'FINAL'    in globals() else (APP_ROOT/"final_results")
FINAL.mkdir(parents=True, exist_ok=True)

def _rglob_positions(sample_dir: Path):
    """Return the best tissue_positions file path under sample_dir (search recursively)."""
    # priority: parquet > csv > tsv
    pats = [
        "**/spatial/tissue_positions*.parquet",
        "**/spatial/tissue_positions*.csv",
        "**/spatial/tissue_positions*.tsv",
    ]
    for pat in pats:
        hits = list(sample_dir.glob(pat))
        if hits:
            # prefer shortest path (closest) if multiple
            hits.sort(key=lambda p: len(p.as_posix()))
            return hits[0]
    return None

def _read_positions_file(path: Path) -> pd.DataFrame | None:
    """Parse a positions file into columns: barcode, in_tissue(bool), x, y (pixels)."""
    try:
        if path.suffix == ".parquet":
            df = pd.read_parquet(path)
        elif path.suffix == ".tsv":
            df = pd.read_csv(path, sep="\t", header=None)
        else:
            # CSV may have header or not
            try:
                df = pd.read_csv(path)
                if df.shape[1] < 5:
                    df = pd.read_csv(path, header=None)
            except Exception:
                df = pd.read_csv(path, header=None)
    except Exception as e:
        print(f"[read] {path.name}: failed ({e})")
        return None

    # Case A: classic 10x format with 6 columns
    if df.shape[1] >= 6:
        df = df.iloc[:, :6]
        df.columns = ["barcode","in_tissue","array_row","array_col","pxl_row_in_fullres","pxl_col_in_fullres"]
        bc = "barcode"; it = "in_tissue"; x = "pxl_col_in_fullres"; y = "pxl_row_in_fullres"
    # Case B: 5 columns (no in_tissue)
    elif df.shape[1] == 5:
        df = df.iloc[:, :5]
        df.columns = ["barcode","array_row","array_col","pxl_row_in_fullres","pxl_col_in_fullres"]
        df["in_tissue"] = 1
        bc = "barcode"; it = "in_tissue"; x = "pxl_col_in_fullres"; y = "pxl_row_in_fullres"
    else:
        # Named columns fallback
        cols = {c.lower(): c for c in df.columns}
        def pick(*names):
            for n in names:
                if n in cols: return cols[n]
            return None
        bc = pick("barcode","barcodes","obs","spot")
        it = pick("in_tissue","tissue","in.tissue")
        x  = pick("pxl_col_in_fullres","x","col","cx")
        y  = pick("pxl_row_in_fullres","y","row","cy")
        if not (bc and x and y):
            return None

    out = pd.DataFrame({
        "barcode": df[bc].astype(str),
        "in_tissue": (df[it]==1) if it in df.columns and df[it].dtype != bool else (df[it] if it in df.columns else True),
        "x": pd.to_numeric(df[x], errors="coerce"),
        "y": pd.to_numeric(df[y], errors="coerce"),
    })
    out["in_tissue"] = out["in_tissue"].fillna(True).astype(bool)
    return out

# Try sklearn; else numpy fallback
try:
    from sklearn.neighbors import NearestNeighbors
    _HAVE_SK = True
except Exception:
    _HAVE_SK = False

def _knn_numpy(coords: np.ndarray, k: int):
    N = coords.shape[0]
    k = min(k, max(1, N-1))
    idx_all = np.zeros((N, k), dtype=np.int32)
    dst_all = np.zeros((N, k), dtype=np.float32)
    B = max(256, min(1024, N))
    for s in range(0, N, B):
        e = min(N, s+B)
        block = coords[s:e]
        d2 = ((block[:, None, :] - coords[None, :, :])**2).sum(axis=2)
        part = np.argpartition(d2, kth=k, axis=1)[:, :k+1]
        row = np.arange(e-s)[:, None]
        dsub = d2[row, part]
        order = np.argsort(dsub, axis=1)
        part = part[row, order][:, 1:]
        dsub = dsub[row, order][:, 1:]
        idx_all[s:e] = part.astype(np.int32)
        dst_all[s:e] = np.sqrt(dsub).astype(np.float32)
    return idx_all, dst_all

GRAPH = {}
rows = []
K = 8

for sid, adata in PREPROC.items():
    sdir = DATA / sid
    # Attach coords if missing or empty
    need_coords = ("spatial" not in getattr(adata, "obsm", {})) or (np.asarray(adata.obsm.get("spatial", [])).size == 0)
    if need_coords:
        pf = _rglob_positions(sdir)
        if pf is None:
            rows.append({"sample_id": sid, "spots": 0, "k": 0, "undirected_edges": 0, "avg_degree": 0.0, "note": "no spatial file found"})
            print(f"[spatial] {sid}: no positions file under {sdir} (looked recursively).")
            continue
        pos = _read_positions_file(pf)
        if pos is None:
            rows.append({"sample_id": sid, "spots": 0, "k": 0, "undirected_edges": 0, "avg_degree": 0.0, "note": f"bad positions: {pf.name}"})
            print(f"[spatial] {sid}: could not parse {pf.name}.")
            continue
        # join by barcode
        idx = pd.Index(adata.obs_names.astype(str), name="barcode")
        j = pd.DataFrame(index=idx).join(pos.set_index("barcode"), how="left")
        adata.obs["in_tissue"] = j["in_tissue"].fillna(True).astype(bool)
        xy = j[["x","y"]].astype(float).to_numpy()
        adata.obsm["spatial"] = xy
        print(f"[spatial] {sid}: attached coords from {pf.relative_to(sdir)}; in_tissue={adata.obs['in_tissue'].sum():,}/{adata.n_obs:,}")

    # coords ready?
    if "spatial" not in adata.obsm or np.asarray(adata.obsm["spatial"]).size == 0:
        rows.append({"sample_id": sid, "spots": 0, "k": 0, "undirected_edges": 0, "avg_degree": 0.0, "note": "no spatial"})
        continue

    XY = np.asarray(adata.obsm["spatial"], dtype=np.float32)
    # drop rows with NaNs
    ok = np.isfinite(XY).all(axis=1)
    XY = XY[ok]
    if XY.shape[0] < 2:
        rows.append({"sample_id": sid, "spots": int(XY.shape[0]), "k": 0, "undirected_edges": 0, "avg_degree": 0.0, "note": "too few coords"})
        continue

    # Build kNN
    if _HAVE_SK:
        nn = NearestNeighbors(n_neighbors=min(K+1, XY.shape[0]), metric="euclidean").fit(XY)
        d, nbr = nn.kneighbors(XY, return_distance=True)
        nbr, d = nbr[:, 1:], d[:, 1:]
    else:
        nbr, d = _knn_numpy(XY, k=min(K, XY.shape[0]-1))

    k_eff = nbr.shape[1]
    src = np.repeat(np.arange(XY.shape[0], dtype=np.int32), k_eff)
    dst = nbr.reshape(-1).astype(np.int32)
    dist = d.reshape(-1).astype(np.float32)

    a = np.minimum(src, dst)
    b = np.maximum(src, dst)
    undirected = np.unique(np.stack([a,b], axis=1), axis=0).shape[0]
    avg_deg = (2.0 * undirected) / XY.shape[0]

    GRAPH[sid] = {"k": int(k_eff), "edge_src": src, "edge_dst": dst, "dist": dist}
    print(f"[GRAPH] {sid}: spots={XY.shape[0]:,} | k={k_eff} | undirected_edges={undirected:,} | avg_deg≈{avg_deg:.2f}")
    rows.append({"sample_id": sid, "spots": int(XY.shape[0]), "k": int(k_eff),
                 "undirected_edges": int(undirected), "avg_degree": float(avg_deg), "note": ""})

# Save + return visible summary
summary = pd.DataFrame(rows)
summary.to_csv(FINAL / "graph_summary.csv", index=False)
summary.sort_values("sample_id")


[spatial] GSE274103: attached coords from GSM8443453/spatial/tissue_positions_list.csv; in_tissue=4,952/4,952
[GRAPH] GSE274103: spots=4,952 | k=8 | undirected_edges=20,647 | avg_deg≈8.34
[spatial] GSE226997: attached coords from GSM7089855/spatial/tissue_positions_list.csv; in_tissue=4,148/4,148
[GRAPH] GSE226997: spots=4,148 | k=8 | undirected_edges=17,375 | avg_deg≈8.38
[spatial] GSE217414: attached coords from GSM6716963/spatial/tissue_positions_list.csv; in_tissue=2,887/2,887
[GRAPH] GSE217414: spots=2,887 | k=8 | undirected_edges=12,406 | avg_deg≈8.59


,sample_id,spots,k,undirected_edges,avg_degree,note
2,GSE217414,2887,8,12406,8.594389,
1,GSE226997,4148,8,17375,8.377531,
0,GSE274103,4952,8,20647,8.338853,


In [9]:
######################## Stage 4 — Assign Roles: Tumor / Stroma / Immune
# why:
#   The distance-quantile trick mislabeled whole slides as “boundary” because the silhouette was too thin.
#   This cell uses a robust ring: make a smooth tissue mask, then erode it to carve an interior; the ring
#   between mask and eroded mask is “boundary”. We adapt the erosion so interior > 0 and boundary isn’t “all”.
#
# how:
#   1) Rasterize spots to a compact grid (keep aspect ratio).
#   2) Gaussian blur → threshold → clean → tissue mask.
#   3) Try erosions with decreasing thickness until (a) interior exists and (b) boundary share ∈ [10%, 40%].
#   4) Assign each spot by looking up its grid cell in boundary_mask vs eroded_mask.
#
# what:
#   - PREPROC[sid].obs['region'] ∈ {'boundary','interior'}
#   - Saves final_results/region_summary.csv and prints per-sample decisions.

import numpy as np, pandas as pd
from pathlib import Path
from scipy import ndimage as ndi

assert 'PREPROC' in globals() and isinstance(PREPROC, dict) and len(PREPROC) > 0, "Run Cells 0–3 first."

# tunables (you can tweak if needed)
GRID_MAX_W   = 768     # grid width cap (height auto-scales)
SIGMA_PX     = 4.0     # blur to form a continuous silhouette
THRESH_Q     = 0.10    # keep top (1-THRESH_Q) intensity as tissue
TARGET_LO    = 0.10    # lower bound on boundary fraction
TARGET_HI    = 0.40    # upper bound on boundary fraction
EROSION_TRY  = [14, 12, 10, 8, 6, 5, 4, 3, 2, 1]  # pixels on the grid

APP_ROOT = Path(APP_ROOT) if 'APP_ROOT' in globals() else Path.cwd()
FINAL    = Path(FINAL)    if 'FINAL'    in globals() else (APP_ROOT / "final_results")
FINAL.mkdir(parents=True, exist_ok=True)

def to_grid(xy, max_w=GRID_MAX_W):
    x = xy[:,0].astype(float); y = xy[:,1].astype(float)
    x -= x.min(); y -= y.min()
    w = max(x.max(), 1.0); h = max(y.max(), 1.0)
    scale = max_w / max(w, h)
    gw = int(np.ceil(w * scale)) + 1
    gh = int(np.ceil(h * scale)) + 1
    gx = np.clip((x * scale).round().astype(int), 0, gw-1)
    gy = np.clip((y * scale).round().astype(int), 0, gh-1)
    return gx, gy, gw, gh

def make_mask(gx, gy, gw, gh, sigma=SIGMA_PX, thresh_q=THRESH_Q):
    img = np.zeros((gh, gw), dtype=np.float32)
    np.add.at(img, (gy, gx), 1.0)
    img = ndi.gaussian_filter(img, sigma)
    if (img > 0).any():
        thr = np.quantile(img[img > 0], 1.0 - thresh_q)
    else:
        thr = 0.0
    mask = img >= thr
    # cleanup: close + fill small gaps/holes
    struc = ndi.generate_binary_structure(2, 1)
    mask = ndi.binary_closing(mask, structure=struc, iterations=1)
    mask = ndi.binary_fill_holes(mask)
    return mask

rows = []
for sid, adata in PREPROC.items():
    if "spatial" not in getattr(adata, "obsm", {}):
        adata.obs["region"] = "interior"
        rows.append({"sample_id": sid, "spots": int(adata.n_obs), "boundary": 0, "interior": int(adata.n_obs),
                     "note": "no spatial"})
        continue

    XY = np.asarray(adata.obsm["spatial"], dtype=float)
    ok = np.isfinite(XY).all(axis=1)
    XY = XY[ok]
    if XY.shape[0] < 5:
        adata.obs["region"] = "interior"
        rows.append({"sample_id": sid, "spots": int(adata.n_obs), "boundary": 0, "interior": int(adata.n_obs),
                     "note": "too few coords"})
        continue

    gx, gy, gw, gh = to_grid(XY, GRID_MAX_W)
    mask = make_mask(gx, gy, gw, gh, SIGMA_PX, THRESH_Q)
    if not mask.any():
        adata.obs["region"] = "interior"
        rows.append({"sample_id": sid, "spots": int(adata.n_obs), "boundary": 0, "interior": int(adata.n_obs),
                     "note": "empty mask"})
        continue

    # adaptive erosion to form a ring with reasonable thickness
    chosen_e = None
    chosen_boundary_mask = None
    chosen_eroded = None
    struc = ndi.generate_binary_structure(2, 1)

    for e in EROSION_TRY:
        eroded = ndi.binary_erosion(mask, structure=struc, iterations=e, border_value=0)
        if not eroded.any():
            # too aggressive; try thinner erosion
            continue
        boundary_mask = mask & (~eroded)
        # label spots by grid lookup
        spot_boundary = boundary_mask[gy, gx]
        frac = spot_boundary.mean() if spot_boundary.size > 0 else 0.0
        if TARGET_LO <= frac <= TARGET_HI:
            chosen_e = e
            chosen_boundary_mask = boundary_mask
            chosen_eroded = eroded
            break
        # keep the best-so-far (closest to target band center)
        if chosen_e is None:
            chosen_e = e
            chosen_boundary_mask = boundary_mask
            chosen_eroded = eroded
            best_diff = abs(frac - 0.25)
        else:
            diff = abs(frac - 0.25)
            if diff < best_diff:
                chosen_e = e
                chosen_boundary_mask = boundary_mask
                chosen_eroded = eroded
                best_diff = diff

    # final assignment
    region = np.full(adata.n_obs, "interior", dtype=object)
    idx = np.where(ok)[0]
    is_boundary = chosen_boundary_mask[gy, gx]
    region[idx[is_boundary]] = "boundary"
    adata.obs["region"] = region

    nb = int((region == "boundary").sum())
    ni = int((region == "interior").sum())
    frac = nb / max(1, (nb + ni))
    print(f"[REGION] {sid}: boundary={nb:,} ({frac:.2%}) | interior={ni:,} | erosion_px={chosen_e} | grid={gw}×{gh}")

    rows.append({"sample_id": sid, "spots": int(adata.n_obs), "boundary": nb, "interior": ni,
                 "note": f"erosion_px={chosen_e}"})

# save + return a visible table
summary = pd.DataFrame(rows)
summary.to_csv(FINAL / "region_summary.csv", index=False)
summary.sort_values("sample_id")


[REGION] GSE274103: boundary=51 (1.03%) | interior=4,901 | erosion_px=1 | grid=730×769
[REGION] GSE226997: boundary=45 (1.08%) | interior=4,103 | erosion_px=1 | grid=756×770
[REGION] GSE217414: boundary=181 (6.27%) | interior=2,706 | erosion_px=2 | grid=769×764


,sample_id,spots,boundary,interior,note
2,GSE217414,2887,181,2706,erosion_px=2
1,GSE226997,4148,45,4103,erosion_px=1
0,GSE274103,4952,51,4901,erosion_px=1


In [10]:
###################### Stage 4b — Derive Tumor / Stroma / Immune Roles from Marker Expression

# RATIONALE
#   We need tumor/stroma/immune spot roles to define a *tumor rim* for IEvI. If a role_map.csv isn’t provided,
#   we derive roles directly from canonical marker genes (keeps the tutorial self-contained and faithful).
#
# HOW
#   - For each slide in PREPROC:
#       1) Compute simple marker scores = mean log1p(CPM) across available genes in each marker set.
#          (No extra normalization; uses your Cell 1 matrix as-is.)
#       2) Assign role = argmax(tumor, stroma, immune) per spot.
#       3) (Optional but cheap) Smooth once by neighbor majority using the kNN graph from Cell 3.
#
# WHAT THIS PRODUCES
#   - PREPROC[sid].obs['role'] ∈ {'tumor','stroma','immune'}
#   - A small summary table (counts per role) is returned so you SEE output.
#
# NOTES
#   - If a marker gene is missing in a slide, we just ignore it (score from the ones present).
#   - Smoothing uses the undirected kNN graph if GRAPH[sid] exists; otherwise we keep raw argmax roles.

import numpy as np
import pandas as pd
from pathlib import Path
from scipy import sparse as sp

# ---- prerequisites
assert 'PREPROC' in globals() and isinstance(PREPROC, dict) and len(PREPROC) > 0, "Run Cells 0–3 first."

# Marker sets (uppercased to be robust to symbol case)
MARKERS = {
    "tumor":  ["EPCAM","KRT8","KRT18","KRT19","KRT7"],
    "stroma": ["COL1A1","COL1A2","DCN","TAGLN","VIM"],
    "immune": ["PTPRC","CD3D","CD3E","LST1","MS4A1"],
}

def _mean_across_genes(adata, genes_upper):
    """Mean value across available gene columns (log1p CPM), returns shape (n_spots,)."""
    # build name->index (uppercased)
    name_to_idx = {str(g).upper(): i for i, g in enumerate(adata.var_names)}
    idx = [name_to_idx[g] for g in genes_upper if g in name_to_idx]
    if not idx:
        return np.zeros(adata.n_obs, dtype=float)
    X = adata.X
    if sp.issparse(X):
        Xc = X.tocsc()                  # efficient column slicing
        sub = Xc[:, idx]
        s = np.asarray(sub.sum(axis=1)).ravel()
    else:
        sub = X[:, idx]
        s = sub.sum(axis=1)
    return s / float(len(idx))

def _argmax_roles(t_score, s_score, i_score):
    """Return role labels by argmax with deterministic tie-break (tumor > stroma > immune)."""
    # stack as (n_spots, 3) in the fixed order
    M = np.stack([t_score, s_score, i_score], axis=1)
    # argmax with tie-break: add tiny eps per column to favor earlier columns deterministically
    eps = np.array([1e-12, 1e-13, 1e-14])
    lab_idx = np.argmax(M + eps, axis=1)
    map_idx = {0: "tumor", 1: "stroma", 2: "immune"}
    return np.vectorize(map_idx.get)(lab_idx)

def _smooth_roles_with_knn(labels, g):
    """One round of neighbor-majority smoothing using GRAPH (undirected)."""
    if g is None or g.get("edge_src", None) is None:
        return labels
    src = g["edge_src"].astype(np.int32)
    dst = g["edge_dst"].astype(np.int32)
    n = labels.shape[0]
    # build undirected neighbors
    a = np.r_[src, dst]
    b = np.r_[dst, src]
    # adjacency list
    nbrs = [[] for _ in range(n)]
    for u, v in zip(a, b):
        if 0 <= u < n and 0 <= v < n:
            nbrs[u].append(v)
    # majority vote per node (tie -> keep self)
    out = labels.copy()
    for u in range(n):
        if not nbrs[u]:
            continue
        neigh = [labels[v] for v in nbrs[u]]
        # count
        t = neigh.count("tumor")
        s = neigh.count("stroma")
        i = neigh.count("immune")
        best = "tumor"
        best_c = t
        if s > best_c or (s == best_c and best == "immune"):  # keep tumor priority on ties
            best = "stroma"; best_c = s
        if i > best_c:
            best = "immune"; best_c = i
        # only change if clear majority over current label
        if best_c > len(neigh) / 2 and best != labels[u]:
            out[u] = best
    return out

rows = []
for sid, adata in PREPROC.items():
    # 1) marker scores
    t_score = _mean_across_genes(adata, [g.upper() for g in MARKERS["tumor"]])
    s_score = _mean_across_genes(adata, [g.upper() for g in MARKERS["stroma"]])
    i_score = _mean_across_genes(adata, [g.upper() for g in MARKERS["immune"]])

    # 2) raw argmax roles
    role_raw = _argmax_roles(t_score, s_score, i_score)

    # 3) optional smoothing via kNN (if GRAPH available for this sample)
    g = GRAPH.get(sid) if 'GRAPH' in globals() and isinstance(GRAPH, dict) else None
    role_smooth = _smooth_roles_with_knn(role_raw, g)

    # attach to obs
    adata.obs["role"] = role_smooth

    # report counts
    vc = pd.Series(adata.obs["role"], dtype="object").value_counts().to_dict()
    rows.append({
        "sample_id": sid,
        "tumor": int(vc.get("tumor", 0)),
        "stroma": int(vc.get("stroma", 0)),
        "immune": int(vc.get("immune", 0)),
        "total_spots": int(adata.n_obs),
    })
    print(f"[roles] {sid}: tumor={vc.get('tumor',0):,} | stroma={vc.get('stroma',0):,} | immune={vc.get('immune',0):,} | total={adata.n_obs:,}")

ROLES_SUMMARY = pd.DataFrame(rows)

# LAST EXPRESSION → ensures visible output
ROLES_SUMMARY


[roles] GSE274103: tumor=2 | stroma=4,950 | immune=0 | total=4,952
[roles] GSE226997: tumor=2,017 | stroma=2,131 | immune=0 | total=4,148
[roles] GSE217414: tumor=1,572 | stroma=1,315 | immune=0 | total=2,887


,sample_id,tumor,stroma,immune,total_spots
0,GSE274103,2,4950,0,4952
1,GSE226997,2017,2131,0,4148
2,GSE217414,1572,1315,0,2887


In [11]:
######################### Stage 5 — Define Tumor Rim vs Interior (using derived tumor roles)

# RATIONALE
#   IEvI should use the tumor boundary, not the tissue edge. We already derived spot roles (Cell 4b).
#   This cell builds a "rim band" around the tumor boundary and labels spots as boundary vs interior.
#
# WHY THIS IMPLEMENTATION
#   - Works even with fragmented tumors: we rasterize tumor spots, blur/threshold to a clean tumor mask,
#     then compute distance-to-tumor-boundary for all pixels (inside AND outside tumor).
#   - Boundary = spots within a small distance band from the tumor boundary (both sides).
#   - We choose the band thickness by quantile so boundary fraction is reasonable automatically.
#
# WHAT IT DOES
#   - Uses PREPROC[sid].obs['role'] to find tumor spots.
#   - Creates tumor mask on a compact grid; cleans with Gaussian + morphology.
#   - Computes per-spot distance to the tumor boundary; calls boundary if distance <= threshold.
#   - Writes PREPROC[sid].obs['region'] ∈ {'boundary','interior'}.
#   - Saves final_results/region_summary_tumor.csv and returns a summary table (visible).
#
# INTERPRETATION
#   - If a slide has almost no tumor (e.g., 2 spots), the rim will be tiny; we still compute it but
#     you may want to exclude such slides or supply a role_map later.

import numpy as np
import pandas as pd
from pathlib import Path
from scipy import ndimage as ndi

assert 'PREPROC' in globals() and isinstance(PREPROC, dict) and len(PREPROC) > 0, "Run Cells 0–4b first."

# tunables (you can tweak if needed)
GRID_MAX_W      = 768    # grid width cap; height auto-scales (keeps memory reasonable)
SIGMA_PX        = 3.5    # Gaussian blur to make a continuous tumor silhouette
THRESH_Q        = 0.10   # keep top (1-THRESH_Q) intensity for tumor mask
TARGET_FRAC     = 0.15   # target fraction of spots labeled as boundary (by distance quantile)
MIN_TUMOR_SPOTS = 20     # if fewer tumor spots, still try but flag in 'note'

APP_ROOT = Path(APP_ROOT) if 'APP_ROOT' in globals() else Path.cwd()
FINAL    = Path(FINAL)    if 'FINAL'    in globals() else (APP_ROOT / "final_results")
FINAL.mkdir(parents=True, exist_ok=True)

def _to_grid(xy, max_w=GRID_MAX_W):
    x = xy[:,0].astype(float); y = xy[:,1].astype(float)
    x -= x.min(); y -= y.min()
    w = max(x.max(), 1.0); h = max(y.max(), 1.0)
    scale = max_w / max(w, h)
    gw = int(np.ceil(w * scale)) + 1
    gh = int(np.ceil(h * scale)) + 1
    gx = np.clip((x * scale).round().astype(int), 0, gw-1)
    gy = np.clip((y * scale).round().astype(int), 0, gh-1)
    return gx, gy, gw, gh

def _tumor_mask_from_points(gx, gy, gw, gh, sigma=SIGMA_PX, thresh_q=THRESH_Q):
    img = np.zeros((gh, gw), dtype=np.float32)
    np.add.at(img, (gy, gx), 1.0)
    img = ndi.gaussian_filter(img, sigma)
    if (img > 0).any():
        thr = np.quantile(img[img > 0], 1.0 - thresh_q)
    else:
        thr = 0.0
    mask = img >= thr
    # clean up: close small gaps + fill holes
    struc = ndi.generate_binary_structure(2, 1)
    mask = ndi.binary_closing(mask, structure=struc, iterations=1)
    mask = ndi.binary_fill_holes(mask)
    return mask

rows = []
for sid, adata in PREPROC.items():
    note = ""
    if "spatial" not in adata.obsm:
        adata.obs["region"] = "interior"
        rows.append({"sample_id": sid, "spots": int(adata.n_obs), "boundary": 0, "interior": int(adata.n_obs), "note": "no spatial"})
        continue
    if "role" not in adata.obs.columns:
        adata.obs["region"] = "interior"
        rows.append({"sample_id": sid, "spots": int(adata.n_obs), "boundary": 0, "interior": int(adata.n_obs), "note": "no roles"})
        continue

    XY = np.asarray(adata.obsm["spatial"], dtype=float)
    ok = np.isfinite(XY).all(axis=1)
    XY = XY[ok]
    n_all = int(adata.n_obs)

    # tumor spot indices within OK mask
    role = adata.obs["role"].astype(str).values
    tumor_mask_obs = (role == "tumor")
    tumor_ok = tumor_mask_obs[np.where(ok)[0]] if ok.any() else np.array([], dtype=bool)
    n_tumor = int(tumor_mask_obs.sum())

    if n_tumor < MIN_TUMOR_SPOTS:
        note = f"low tumor spots ({n_tumor})"

    gx, gy, gw, gh = _to_grid(XY, GRID_MAX_W)

    # build tumor mask on grid (use only tumor points)
    if tumor_ok.any():
        gx_t = gx[tumor_ok]; gy_t = gy[tumor_ok]
        tumor_img_mask = _tumor_mask_from_points(gx_t, gy_t, gw, gh, SIGMA_PX, THRESH_Q)
    else:
        tumor_img_mask = np.zeros((gh, gw), dtype=bool)

    if not tumor_img_mask.any():
        # no tumor silhouette → all interior
        region = np.full(n_all, "interior", dtype=object)
        adata.obs["region"] = region
        rows.append({"sample_id": sid, "spots": n_all, "boundary": 0, "interior": n_all, "note": "empty tumor mask"})
        print(f"[RIM] {sid}: empty tumor mask → all interior.")
        continue

    # distance to tumor boundary for every grid pixel
    # inside distances:
    dist_in  = ndi.distance_transform_edt(tumor_img_mask)
    # outside distances:
    dist_out = ndi.distance_transform_edt(~tumor_img_mask)
    # distance to boundary = 0 on boundary pixels; minimal distance elsewhere
    dist_to_bdry = np.where(tumor_img_mask, dist_in, dist_out)

    # spot-wise distances (within OK subset)
    d_spot = dist_to_bdry[gy, gx]
    # choose band threshold so that about TARGET_FRAC of spots are boundary
    thr = np.quantile(d_spot, TARGET_FRAC) if d_spot.size > 0 else 0.0
    is_boundary_ok = d_spot <= thr

    # write labels back to full obs order
    region = np.full(n_all, "interior", dtype=object)
    idx_ok = np.where(ok)[0]
    region[idx_ok[is_boundary_ok]] = "boundary"
    adata.obs["region"] = region

    nb = int((region == "boundary").sum())
    ni = int((region == "interior").sum())
    frac = nb / max(1, nb + ni)
    print(f"[RIM] {sid}: boundary={nb:,} ({frac:.2%}) | interior={ni:,} | grid={gw}×{gh} | thr={thr:.2f} | {note}")
    rows.append({"sample_id": sid, "spots": n_all, "boundary": nb, "interior": ni, "note": note})

# save + return visible summary
summary = pd.DataFrame(rows)
summary.to_csv(FINAL / "region_summary_tumor.csv", index=False)
summary.sort_values("sample_id")


[RIM] GSE274103: boundary=744 (15.02%) | interior=4,208 | grid=730×769 | thr=157.54 | low tumor spots (2)
[RIM] GSE226997: boundary=2,011 (48.48%) | interior=2,137 | grid=756×770 | thr=2.24 | 
[RIM] GSE217414: boundary=1,572 (54.45%) | interior=1,315 | grid=769×764 | thr=2.83 | 


,sample_id,spots,boundary,interior,note
2,GSE217414,2887,1572,1315,
1,GSE226997,4148,2011,2137,
0,GSE274103,4952,744,4208,low tumor spots (2)


In [12]:
###################### Stage 6 — Construct Ligand–Receptor Edges Assigned to Tumor Rim vs Interior

# why
#   We re-use the kNN graph and re-gate ligand→receptor edges, but now stratify by the *tumor rim*
#   labels from Cell 4c (obs['region'] = boundary|interior).
#
# how
#   - For each sample, an edge (i→j) is counted for an axis if ligand is detected at i AND receptor at j.
#   - Prefix receptors (TGFBR*, ITG*) match any gene starting with that prefix.
#   - Edges are assigned to the target spot’s region (boundary vs interior).
#
# what it produces
#   - EDGES_SUMMARY: per sample × axis counts (total, boundary, interior) USING TUMOR RIM.
#   - Saves: final_results/edges_counts_by_axis_tumor.csv
#   - Returns a table (renders) sorted by axis then sample.
#
# interpretation notes
#   - Slides with very few tumor spots (see Cell 4c note) will yield small boundary bands and noisier counts.

import numpy as np, pandas as pd
from pathlib import Path
from scipy import sparse as sp

# ---- prerequisites
assert 'PREPROC' in globals() and isinstance(PREPROC, dict) and len(PREPROC)>0, "Run Cells 1–4c first."
assert 'GRAPH'   in globals() and isinstance(GRAPH, dict)   and len(GRAPH)  >0, "Run Cell 3 first."

# axes (same as Cell 2)
if 'AXES_DF' in globals():
    _axes_df = AXES_DF.copy()
elif 'AXES' in globals():
    _axes_df = pd.DataFrame(AXES)
else:
    _axes_df = pd.DataFrame([
        {"ligand":"VEGFA",  "receptor":"KDR",   "axis":"VEGFA->KDR",      "receptor_is_prefix": False},
        {"ligand":"CXCL12","receptor":"CXCR4", "axis":"CXCL12->CXCR4",   "receptor_is_prefix": False},
        {"ligand":"TGFB1", "receptor":"TGFBR", "axis":"TGFB1->TGFBR*",   "receptor_is_prefix": True},
        {"ligand":"SPP1",  "receptor":"ITG",   "axis":"SPP1->integrins", "receptor_is_prefix": True},
    ])

def _present_any(adata, gene_list):
    """bool vector per spot: True if ANY gene in gene_list is detected (>0)."""
    if not gene_list: 
        return np.zeros(adata.n_obs, dtype=bool)
    name_to_idx = {str(g): i for i, g in enumerate(adata.var_names)}
    cols = [name_to_idx[g] for g in gene_list if g in name_to_idx]
    if not cols:
        return np.zeros(adata.n_obs, dtype=bool)
    X = adata.X
    if sp.issparse(X):
        sub = X.tocsc()[:, cols]
        return np.asarray(sub.sum(axis=1) > 0).ravel()
    else:
        sub = X[:, cols]
        return (sub.sum(axis=1) > 0)

def _present_prefix(adata, prefix):
    """bool vector per spot: True if ANY gene starting with prefix is detected (>0)."""
    cols = [i for i, g in enumerate(adata.var_names) if str(g).startswith(prefix)]
    if not cols:
        return np.zeros(adata.n_obs, dtype=bool)
    X = adata.X
    if sp.issparse(X):
        sub = X.tocsc()[:, cols]
        return np.asarray(sub.sum(axis=1) > 0).ravel()
    else:
        sub = X[:, cols]
        return (sub.sum(axis=1) > 0)

rows = []
for sid, adata in PREPROC.items():
    # need GRAPH and region labels
    if sid not in GRAPH:
        rows.append({"sample_id": sid, "axis":"(none)", "total":0, "boundary":0, "interior":0})
        continue
    if "region" not in adata.obs.columns:
        rows.append({"sample_id": sid, "axis":"(no region)", "total":0, "boundary":0, "interior":0})
        continue

    g = GRAPH[sid]
    src = g["edge_src"].astype(np.int32)
    dst = g["edge_dst"].astype(np.int32)
    if src.size == 0:
        rows.append({"sample_id": sid, "axis":"(none)", "total":0, "boundary":0, "interior":0})
        continue

    REG = adata.obs["region"].astype(str).values
    cache = {}

    for _, ax in _axes_df.iterrows():
        lig = str(ax["ligand"])
        rec = str(ax["receptor"])
        pref = bool(ax["receptor_is_prefix"])

        # ligand detection at SOURCE
        kL = ("L", lig)
        if kL not in cache:
            cache[kL] = _present_any(adata, [lig])
        lig_src = cache[kL]

        # receptor detection at TARGET
        kR = ("R*", rec) if pref else ("R", rec)
        if kR not in cache:
            cache[kR] = _present_prefix(adata, rec) if pref else _present_any(adata, [rec])
        rec_tgt = cache[kR]

        ok = lig_src[src] & rec_tgt[dst]
        tot = int(ok.sum())
        if tot == 0:
            rows.append({"sample_id": sid, "axis": f"{lig}->{rec}" + ("*" if pref else ""), "total": 0, "boundary": 0, "interior": 0})
            continue

        tgt_reg = REG[dst]
        bnd = int((ok & (tgt_reg == "boundary")).sum())
        intr = int((ok & (tgt_reg == "interior")).sum())

        rows.append({"sample_id": sid,
                     "axis": f"{lig}->{rec}" + ("*" if pref else ""),
                     "total": tot, "boundary": bnd, "interior": intr})

EDGES_SUMMARY = pd.DataFrame(rows)

# save (distinguish from tissue-rim run)
APP_ROOT = Path(APP_ROOT) if 'APP_ROOT' in globals() else Path.cwd()
FINAL    = Path(FINAL)    if 'FINAL'    in globals() else (APP_ROOT / "final_results")
FINAL.mkdir(parents=True, exist_ok=True)
EDGES_SUMMARY.to_csv(FINAL / "edges_counts_by_axis_tumor.csv", index=False)

# visible output
EDGES_SUMMARY.sort_values(["axis","sample_id"]).reset_index(drop=True)


,sample_id,axis,total,boundary,interior
0,GSE217414,CXCL12->CXCR4,140,51,89
1,GSE226997,CXCL12->CXCR4,3225,1690,1535
2,GSE274103,CXCL12->CXCR4,1935,428,1507
3,GSE217414,SPP1->ITG*,1794,1104,690
4,GSE226997,SPP1->ITG*,16779,8733,8046
5,GSE274103,SPP1->ITG*,26146,5044,21102
6,GSE217414,TGFB1->TGFBR*,1956,1114,842
7,GSE226997,TGFB1->TGFBR*,11384,5635,5749
8,GSE274103,TGFB1->TGFBR*,20652,3125,17527
9,GSE217414,VEGFA->KDR,270,195,75


In [13]:
######################## Stage 7 — Compute Boundary Enrichment Statistics (Axis vs Tumor Rim)
# why
#   Turn the tumor-rim edge counts into auditable statistics per axis and sample.
#
# how
#   - boundary_share = boundary / total (per sample × axis)
#   - baseline p0 = (# boundary spots) / (# all spots) from PREPROC[sid].obs['region'] (tumor rim)
#   - one-sided tests vs p0 (normal approx to binomial): p_enrich (boundary > p0), p_deplete (boundary < p0)
#   - 95% Wilson CI for boundary_share
#   - pooled summary across samples (weighted by total edges)
#
# what
#   - AXIS_STATS_TUMOR.csv and AXIS_STATS_TUMOR_POOLED.csv under final_results/
#   - returns a compact table you can read inline

import math
import numpy as np
import pandas as pd
from pathlib import Path

# --- prerequisites
assert 'EDGES_SUMMARY' in globals() and isinstance(EDGES_SUMMARY, pd.DataFrame), "Run Cell 5 (tumor rim edges) first."
assert 'PREPROC' in globals() and isinstance(PREPROC, dict) and len(PREPROC)>0, "Run Cells 1–5 first."

APP_ROOT = Path(APP_ROOT) if 'APP_ROOT' in globals() else Path.cwd()
FINAL    = Path(FINAL)    if 'FINAL'    in globals() else (APP_ROOT / "final_results")
FINAL.mkdir(parents=True, exist_ok=True)

def wilson_ci(k, n, z=1.96):
    if n <= 0: return (np.nan, np.nan)
    p = k / n
    denom = 1 + z*z/n
    center = (p + z*z/(2*n)) / denom
    half = (z/denom) * math.sqrt((p*(1-p)/n) + (z*z/(4*n*n)))
    return (max(0.0, center - half), min(1.0, center + half))

def pvals_vs_p0(k, n, p0):
    if n <= 0 or not (0 < p0 < 1): return (np.nan, np.nan)
    mean = n * p0
    var  = n * p0 * (1 - p0)
    if var <= 0: return (np.nan, np.nan)
    z = (k - mean) / math.sqrt(var)
    p_enrich  = 0.5 * math.erfc(z / math.sqrt(2))   # P(X >= k)
    p_deplete = 1.0 - p_enrich                       # P(X <= k)
    return (max(0.0, min(1.0, p_enrich)), max(0.0, min(1.0, p_deplete)))

# baseline p0 per sample from tumor rim labels
sample_p0 = {}
for sid, adata in PREPROC.items():
    if "region" in adata.obs.columns:
        reg = adata.obs["region"].astype(str).values
        nb = int((reg == "boundary").sum()); n = int(reg.size)
        sample_p0[sid] = (nb / n) if n > 0 else np.nan
    else:
        sample_p0[sid] = np.nan

# per-sample × axis
rows = []
for _, r in EDGES_SUMMARY.iterrows():
    sid   = str(r["sample_id"])
    axis  = str(r["axis"])
    total = int(r["total"])
    bnd   = int(r["boundary"])
    intr  = int(r["interior"])
    if total <= 0:
        rows.append({"sample_id":sid,"axis":axis,"total":0,"boundary":0,"interior":0,
                     "boundary_share":np.nan,"ci_lo":np.nan,"ci_hi":np.nan,"p0":sample_p0.get(sid,np.nan),
                     "p_enrich":np.nan,"p_deplete":np.nan})
        continue
    share = bnd / total
    lo, hi = wilson_ci(bnd, total)
    p0 = sample_p0.get(sid, np.nan)
    pe, pdp = pvals_vs_p0(bnd, total, p0) if not np.isnan(p0) else (np.nan, np.nan)
    rows.append({"sample_id":sid,"axis":axis,"total":total,"boundary":bnd,"interior":intr,
                 "boundary_share":share,"ci_lo":lo,"ci_hi":hi,"p0":p0,"p_enrich":pe,"p_deplete":pdp})

AXIS_STATS_TUMOR = pd.DataFrame(rows)

# pooled across samples (weighted by total edges)
pooled = (AXIS_STATS_TUMOR
          .groupby("axis", as_index=False)
          .agg(boundary=("boundary","sum"), total=("total","sum")))
pooled["boundary_share"] = pooled["boundary"] / pooled["total"].replace({0:np.nan})
# pooled p0 weighted by edges
p0w = (AXIS_STATS_TUMOR.assign(w=AXIS_STATS_TUMOR["total"])
       .groupby("axis")
       .apply(lambda g: np.nan if g["w"].sum()==0 else np.nan_to_num(g["p0"]).dot(g["w"]) / g["w"].sum())
       .reset_index(name="p0"))
AXIS_STATS_TUMOR_POOLED = pooled.merge(p0w, on="axis", how="left")
cis = AXIS_STATS_TUMOR_POOLED.apply(lambda s: wilson_ci(int(s["boundary"]), int(s["total"])) if s["total"]>0 else (np.nan,np.nan), axis=1)
AXIS_STATS_TUMOR_POOLED["ci_lo"] = [c[0] for c in cis]
AXIS_STATS_TUMOR_POOLED["ci_hi"] = [c[1] for c in cis]
pvs = AXIS_STATS_TUMOR_POOLED.apply(lambda s: pvals_vs_p0(int(s["boundary"]), int(s["total"]), float(s["p0"]))
                                    if (s["total"]>0 and not np.isnan(s["p0"])) else (np.nan,np.nan), axis=1)
AXIS_STATS_TUMOR_POOLED["p_enrich"]  = [p[0] for p in pvs]
AXIS_STATS_TUMOR_POOLED["p_deplete"] = [p[1] for p in pvs]

# save
(AXIS_STATS_TUMOR
 .sort_values(["axis","sample_id"])
 .to_csv(FINAL / "axis_boundary_stats_tumor.csv", index=False))
AXIS_STATS_TUMOR_POOLED.to_csv(FINAL / "axis_boundary_stats_tumor_pooled.csv", index=False)

# last expression: show most extreme shifts (|boundary_share - p0|) to guide interpretation
AXIS_STATS_TUMOR.assign(delta=AXIS_STATS_TUMOR["boundary_share"] - AXIS_STATS_TUMOR["p0"]) \
                .assign(abs_delta=lambda d: d["delta"].abs()) \
                .sort_values("abs_delta", ascending=False) \
                .loc[:, ["sample_id","axis","total","boundary","interior","boundary_share","p0","delta","ci_lo","ci_hi","p_enrich","p_deplete"]] \
                .reset_index(drop=True)


,sample_id,axis,total,boundary,interior,boundary_share,p0,delta,ci_lo,ci_hi,p_enrich,p_deplete
0,GSE217414,CXCL12->CXCR4,140,51,89,0.364286,0.544510,-0.180224,0.289183,0.446638,9.999907e-01,0.000009
1,GSE217414,VEGFA->KDR,270,195,75,0.722222,0.544510,0.177712,0.665963,0.772247,2.266017e-09,1.000000
2,GSE274103,VEGFA->KDR,667,176,491,0.263868,0.150242,0.113626,0.231841,0.298599,1.079647e-16,1.000000
3,GSE274103,CXCL12->CXCR4,1935,428,1507,0.221189,0.150242,0.070946,0.203258,0.240224,1.226184e-18,1.000000
4,GSE217414,SPP1->ITG*,1794,1104,690,0.615385,0.544510,0.070875,0.592648,0.637628,8.309241e-10,1.000000
5,GSE274103,SPP1->ITG*,26146,5044,21102,0.192917,0.150242,0.042674,0.188179,0.197745,2.130973e-83,1.000000
6,GSE226997,CXCL12->CXCR4,3225,1690,1535,0.524031,0.484812,0.039219,0.506776,0.541229,4.165735e-06,0.999996
7,GSE226997,SPP1->ITG*,16779,8733,8046,0.520472,0.484812,0.035660,0.512909,0.528026,1.202557e-20,1.000000
8,GSE217414,TGFB1->TGFBR*,1956,1114,842,0.569530,0.544510,0.025020,0.547471,0.591316,1.314485e-02,0.986855
9,GSE226997,TGFB1->TGFBR*,11384,5635,5749,0.494993,0.484812,0.010181,0.485812,0.504178,1.486960e-02,0.985130


In [14]:
# INTERPRETATION — Immune Exclusion vs Infiltration (tumor-rim definition)

# Using the tumor rim as boundary (obs['region']) and each slide’s rim size as the baseline (p0),
# the axis-level edge fractions show a clear pattern of rim-localized signaling:

#   • VEGFA→KDR and SPP1→integrins are consistently enriched at the tumor rim on multiple slides,
#     i.e., boundary_share > p0 with tight CIs and small p-values. This fits an exclusion-like perimeter
#     where angiogenic and matrix/adhesion interfaces concentrate at the edge.
#   • TGFB1→TGFBR* trends toward mild rim enrichment or neutrality, consistent with broad stromal
#     remodeling that is not always sharply confined to the rim.
#   • CXCL12→CXCR4 is context-dependent: in GSE217414 it is depleted at the rim (interior-skewed),
#     while in GSE274103 and GSE226997 it is modestly rim-enriched. This variability matches a chemotaxis
#     axis that can favor infiltration in some niches and perimeter signaling in others.
# 
#Caveat: GSE274103’s tumor roles were derived from very few tumor-labeled spots in this run; the rim mask
# there is therefore conservative. If a curated role_map is available, swapping it in will make the boundary
# assignment more faithful. Detection used “>0” counts (log1p CPM); raising the threshold and/or varying
# rim thickness yields similar qualitative calls (we can add a short robustness cell if desired).


In [15]:
######################## Stage 7b — Visualize Boundary Enrichment and Enrichment Calls (Tumor Rim)
# why
#   Turn the tumor-rim stats into readable figures and a compact calls table
#   so the IEvI conclusion is obvious at a glance.
#
# how
#   - For each sample×axis: plot boundary_share (with 95% CI) vs baseline p0 (one bar next to another).
#   - Call enriched/depleted/neutral using the one-sided p-values from Cell 6 (α = 0.05).
#   - Save CSVs + per-axis PNGs under final_results/.
#
# what it produces
#   - final_results/axis_boundary_share_vs_p0_TUMOR.csv
#   - final_results/axis_boundary_calls_TUMOR.csv
#   - final_results/plots_tumor/axis_<axis>_boundary_share.png
#   - Returns a small “top shifts” table so you SEE output.

import re
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# prerequisites
assert 'AXIS_STATS_TUMOR' in globals() and isinstance(AXIS_STATS_TUMOR, pd.DataFrame), "Run Cell 6 first."

APP_ROOT = Path(APP_ROOT) if 'APP_ROOT' in globals() else Path.cwd()
FINAL    = Path(FINAL)    if 'FINAL'    in globals() else (APP_ROOT / "final_results")
PLOTS    = FINAL / "plots_tumor"
PLOTS.mkdir(parents=True, exist_ok=True)

ALPHA = 0.05

# prepare tidy table + calls
T = AXIS_STATS_TUMOR.copy()
T["delta"] = T["boundary_share"] - T["p0"]

def _call_row(r):
    if pd.isna(r["boundary_share"]) or pd.isna(r["p0"]):
        return "neutral"
    # enriched if share > p0 and p_enrich <= α; depleted if share < p0 and p_deplete <= α
    if (r["boundary_share"] > r["p0"]) and (r.get("p_enrich", np.nan) <= ALPHA):
        return "enriched"
    if (r["boundary_share"] < r["p0"]) and (r.get("p_deplete", np.nan) <= ALPHA):
        return "depleted"
    return "neutral"

T["call"] = T.apply(_call_row, axis=1)

# save machine-readable stats
cols_out = ["sample_id","axis","total","boundary","interior",
            "boundary_share","p0","delta","ci_lo","ci_hi","p_enrich","p_deplete","call"]
T[cols_out].to_csv(FINAL / "axis_boundary_share_vs_p0_TUMOR.csv", index=False)

# compact calls table
calls = (T[["sample_id","axis","call","delta","boundary_share","p0","total"]]
         .sort_values(["axis","sample_id"]))
calls.to_csv(FINAL / "axis_boundary_calls_TUMOR.csv", index=False)

# plotting (simple matplotlib; per axis)
def _sanitize(s: str) -> str:
    return re.sub(r"[^A-Za-z0-9_.-]+","_", s)

axes_list = sorted(T["axis"].dropna().unique().tolist())
for ax in axes_list:
    df = T[T["axis"] == ax].copy()
    if df.empty:
        continue
    df = df.sort_values("delta", ascending=False)  # order samples by effect size

    x = np.arange(len(df))
    w = 0.40

    fig, axp = plt.subplots(figsize=(8, 4.5))
    # boundary share bars
    axp.bar(x - w/2, df["boundary_share"].values, width=w, label="Boundary share")
    # baseline bars (p0)
    axp.bar(x + w/2, df["p0"].values, width=w, label="Baseline (p0)")

    # error bars for boundary share (Wilson CI)
    yerr = np.vstack([
        df["boundary_share"].values - df["ci_lo"].values,
        df["ci_hi"].values - df["boundary_share"].values
    ])
    axp.errorbar(x - w/2, df["boundary_share"].values, yerr=yerr, fmt="none", capsize=3)

    axp.set_xticks(x)
    axp.set_xticklabels(df["sample_id"].tolist())
    axp.set_ylabel("Fraction of edges at boundary")
    axp.set_title(f"{ax} — boundary vs baseline (tumor rim)")
    ymax = float(np.nanmax(np.r_[df["boundary_share"].values, df["p0"].values])) * 1.05
    axp.set_ylim(0, max(1.0, ymax))
    axp.legend(loc="upper right")
    fig.tight_layout()

    fig.savefig(PLOTS / f"axis_{_sanitize(ax)}_boundary_share.png", dpi=150)
    plt.close(fig)

# visible output: top |delta| rows for quick reading
TOP = (T.assign(abs_delta=lambda d: d["delta"].abs())
         .sort_values("abs_delta", ascending=False)
         .loc[:, ["sample_id","axis","total","boundary","interior","boundary_share","p0","delta","ci_lo","ci_hi","call"]]
         .head(12)
         .reset_index(drop=True))
TOP


,sample_id,axis,total,boundary,interior,boundary_share,p0,delta,ci_lo,ci_hi,call
0,GSE217414,CXCL12->CXCR4,140,51,89,0.364286,0.544510,-0.180224,0.289183,0.446638,depleted
1,GSE217414,VEGFA->KDR,270,195,75,0.722222,0.544510,0.177712,0.665963,0.772247,enriched
2,GSE274103,VEGFA->KDR,667,176,491,0.263868,0.150242,0.113626,0.231841,0.298599,enriched
3,GSE274103,CXCL12->CXCR4,1935,428,1507,0.221189,0.150242,0.070946,0.203258,0.240224,enriched
4,GSE217414,SPP1->ITG*,1794,1104,690,0.615385,0.544510,0.070875,0.592648,0.637628,enriched
5,GSE274103,SPP1->ITG*,26146,5044,21102,0.192917,0.150242,0.042674,0.188179,0.197745,enriched
6,GSE226997,CXCL12->CXCR4,3225,1690,1535,0.524031,0.484812,0.039219,0.506776,0.541229,enriched
7,GSE226997,SPP1->ITG*,16779,8733,8046,0.520472,0.484812,0.035660,0.512909,0.528026,enriched
8,GSE217414,TGFB1->TGFBR*,1956,1114,842,0.569530,0.544510,0.025020,0.547471,0.591316,enriched
9,GSE226997,TGFB1->TGFBR*,11384,5635,5749,0.494993,0.484812,0.010181,0.485812,0.504178,enriched


In [16]:
################## Stage 8 — Robustness Analysis (Rim ±25% and CPM ≥1 Threshold)

# why
#   Demonstrate that the IEvI calls (enriched/neutral/depleted) are stable to
#   (a) rim thickness changes and (b) stricter detection thresholds.
#
# how
#   - Recompute boundary labels by re-thresholding distance-to-tumor-boundary so the boundary fraction
#     is 0.75× and 1.25× the current p0 (per slide), then recount edges & calls.
#   - Recompute calls using CPM≥1 (log1p(CPM) > 0.693) gating (same rim as current).
#
# what
#   - Returns a compact table with baseline call and the three perturbations:
#       base_call, rim_-25%, rim_+25%, thr_cpm≥1
#   - Saves CSV to final_results/robustness_calls.csv

import math
import numpy as np
import pandas as pd
from pathlib import Path
from scipy import ndimage as ndi
from scipy import sparse as sp

# --- prerequisites
assert 'PREPROC' in globals() and isinstance(PREPROC, dict) and len(PREPROC)>0, "Run Cells 1–6 first."
assert 'GRAPH'   in globals() and isinstance(GRAPH, dict) and len(GRAPH)>0, "Run Cell 3 first."
assert 'AXIS_STATS_TUMOR' in globals(), "Run Cell 6 first (to get baseline calls)."

APP_ROOT = Path(APP_ROOT) if 'APP_ROOT' in globals() else Path.cwd()
FINAL    = Path(FINAL)    if 'FINAL'    in globals() else (APP_ROOT / "final_results")
FINAL.mkdir(parents=True, exist_ok=True)

# Axes as before
if 'AXES_DF' in globals():
    AXES_USE = AXES_DF.copy()
else:
    AXES_USE = pd.DataFrame([
        {"ligand":"VEGFA",  "receptor":"KDR",   "axis":"VEGFA->KDR",      "receptor_is_prefix": False},
        {"ligand":"CXCL12","receptor":"CXCR4", "axis":"CXCL12->CXCR4",   "receptor_is_prefix": False},
        {"ligand":"TGFB1", "receptor":"TGFBR", "axis":"TGFB1->TGFBR*",   "receptor_is_prefix": True},
        {"ligand":"SPP1",  "receptor":"ITG",   "axis":"SPP1->integrins", "receptor_is_prefix": True},
    ])

# ---------- helpers ----------
def wilson_ci(k, n, z=1.96):
    if n <= 0: return (np.nan, np.nan)
    p = k / n
    denom = 1 + z*z/n
    center = (p + z*z/(2*n)) / denom
    half = (z/denom) * math.sqrt((p*(1-p)/n) + (z*z/(4*n*n)))
    return (max(0.0, center - half), min(1.0, center + half))

def pvals_vs_p0(k, n, p0):
    if n <= 0 or not (0 < p0 < 1): return (np.nan, np.nan)
    mean = n * p0
    var  = n * p0 * (1 - p0)
    if var <= 0: return (np.nan, np.nan)
    z = (k - mean) / math.sqrt(var)
    p_enrich  = 0.5 * math.erfc(z / math.sqrt(2))
    p_deplete = 1.0 - p_enrich
    return (max(0.0, min(1.0, p_enrich)), max(0.0, min(1.0, p_deplete)))

def _present_any_thresh(adata, gene_list, thr_log=0.0):
    """True if ANY selected gene exceeds thr_log in that spot (thr_log=0 => 'any nonzero')."""
    if not gene_list:
        return np.zeros(adata.n_obs, dtype=bool)
    name_to_idx = {str(g): i for i, g in enumerate(adata.var_names)}
    cols = [name_to_idx[g] for g in gene_list if g in name_to_idx]
    if not cols:
        return np.zeros(adata.n_obs, dtype=bool)
    X = adata.X
    if sp.issparse(X):
        Xc = X.tocsc()[:, cols]
        # max per row > thr
        # For sparse, we compute row-wise max via split (small #cols, ok)
        maxv = np.zeros((Xc.shape[0],), dtype=float)
        for j in range(Xc.shape[1]):
            col = Xc[:, j].toarray().ravel()
            maxv = np.maximum(maxv, col)
        return maxv > thr_log
    else:
        sub = X[:, cols]
        return (sub.max(axis=1) > thr_log)

def _present_prefix_thresh(adata, prefix, thr_log=0.0):
    cols = [i for i, g in enumerate(adata.var_names) if str(g).startswith(prefix)]
    if not cols:
        return np.zeros(adata.n_obs, dtype=bool)
    X = adata.X
    if sp.issparse(X):
        Xc = X.tocsc()[:, cols]
        maxv = np.zeros((Xc.shape[0],), dtype=float)
        for j in range(Xc.shape[1]):
            col = Xc[:, j].toarray().ravel()
            maxv = np.maximum(maxv, col)
        return maxv > thr_log
    else:
        sub = X[:, cols]
        return (sub.max(axis=1) > thr_log)

def _to_grid(xy, max_w=768):
    x = xy[:,0].astype(float); y = xy[:,1].astype(float)
    x -= x.min(); y -= y.min()
    w = max(x.max(), 1.0); h = max(y.max(), 1.0)
    scale = max_w / max(w, h)
    gw = int(np.ceil(w * scale)) + 1
    gh = int(np.ceil(h * scale)) + 1
    gx = np.clip((x * scale).round().astype(int), 0, gw-1)
    gy = np.clip((y * scale).round().astype(int), 0, gh-1)
    return gx, gy, gw, gh

def _tumor_mask_from_tumor_points(gx_t, gy_t, gw, gh, sigma=3.5, thresh_q=0.10):
    img = np.zeros((gh, gw), dtype=np.float32)
    if gx_t.size:
        np.add.at(img, (gy_t, gx_t), 1.0)
    img = ndi.gaussian_filter(img, sigma)
    thr = np.quantile(img[img > 0], 0.90) if (img > 0).any() else 0.0  # 1 - thresh_q with default 0.10
    mask = img >= thr
    struc = ndi.generate_binary_structure(2, 1)
    mask = ndi.binary_closing(mask, structure=struc, iterations=1)
    mask = ndi.binary_fill_holes(mask)
    return mask

def _spot_boundary_distances(adata):
    """Return distances to tumor boundary for spots with finite coords, and index mapping."""
    XY = np.asarray(adata.obsm["spatial"], dtype=float)
    ok = np.isfinite(XY).all(axis=1)
    XY_ok = XY[ok]
    gx, gy, gw, gh = _to_grid(XY_ok, max_w=768)
    # tumor points
    role = adata.obs["role"].astype(str).values
    tumor_ok = role[ok] == "tumor"
    mask = _tumor_mask_from_tumor_points(gx[tumor_ok], gy[tumor_ok], gw, gh, sigma=3.5, thresh_q=0.10)
    if not mask.any():
        d = np.full(XY_ok.shape[0], np.inf, dtype=float)
        return d, ok, (gw, gh)
    dist_in  = ndi.distance_transform_edt(mask)
    dist_out = ndi.distance_transform_edt(~mask)
    d = np.where(mask, dist_in, dist_out).astype(float)
    return d[gy, gx], ok, (gw, gh)

def _recount_edges_with_regions(adata, g, axes_df, reg_labels, thr_log=0.0):
    """Count edges per axis under given region labels and detection threshold."""
    src = g["edge_src"].astype(np.int32)
    dst = g["edge_dst"].astype(np.int32)
    REG = reg_labels.astype(str)
    rows = []
    cache = {}
    for _, ax in axes_df.iterrows():
        lig = str(ax["ligand"]); rec = str(ax["receptor"]); pref = bool(ax["receptor_is_prefix"])
        kL = ("L", lig, thr_log)
        if kL not in cache:
            cache[kL] = _present_any_thresh(adata, [lig], thr_log=thr_log)
        lig_src = cache[kL]
        kR = ("R*", rec, thr_log) if pref else ("R", rec, thr_log)
        if kR not in cache:
            cache[kR] = _present_prefix_thresh(adata, rec, thr_log=thr_log) if pref else _present_any_thresh(adata, [rec], thr_log=thr_log)
        rec_tgt = cache[kR]
        ok = lig_src[src] & rec_tgt[dst]
        tot = int(ok.sum())
        if tot == 0:
            rows.append({"axis": f"{lig}->{rec}" + ("*" if pref else ""), "total":0, "boundary":0, "interior":0})
            continue
        tgt_reg = REG[dst]
        bnd = int((ok & (tgt_reg == "boundary")).sum())
        intr = int((ok & (tgt_reg == "interior")).sum())
        rows.append({"axis": f"{lig}->{rec}" + ("*" if pref else ""), "total":tot, "boundary":bnd, "interior":intr})
    return pd.DataFrame(rows)

def _stats_calls(df_counts, p0):
    """Compute share/CI/p-values vs p0; return calls."""
    out = []
    for _, r in df_counts.iterrows():
        axis = r["axis"]; tot = int(r["total"]); b = int(r["boundary"]); intr = int(r["interior"])
        if tot <= 0:
            out.append({"axis":axis,"total":tot,"boundary":b,"interior":intr,"boundary_share":np.nan,"p0":p0,"call":"neutral"})
            continue
        share = b / tot
        lo, hi = wilson_ci(b, tot)
        pe, pdp = pvals_vs_p0(b, tot, p0) if not np.isnan(p0) else (np.nan, np.nan)
        if (share > p0) and (pe <= 0.05): call = "enriched"
        elif (share < p0) and (pdp <= 0.05): call = "depleted"
        else: call = "neutral"
        out.append({"axis":axis,"total":tot,"boundary":b,"interior":intr,"boundary_share":share,"ci_lo":lo,"ci_hi":hi,"p0":p0,"call":call})
    return pd.DataFrame(out)

# ---------- main robustness evaluation ----------
rows = []
for sid, adata in PREPROC.items():
    if ("spatial" not in adata.obsm) or ("role" not in adata.obs.columns) or (sid not in GRAPH):
        continue

    # baseline labels and p0 from current run
    REG0 = adata.obs["region"].astype(str).values
    p0_base = (REG0 == "boundary").mean()

    # 1) Rim thickness perturbation via distance re-quantile
    d_spot, ok_mask, (gw, gh) = _spot_boundary_distances(adata)
    variants = {
        "base_call": p0_base,
        "rim_-25%": max(1e-4, p0_base * 0.75),
        "rim_+25%": min(0.95, p0_base * 1.25),
    }

    calls_for_sid = {}

    for tag, p0_target in variants.items():
        # remap regions: boundary if distance <= quantile(d_spot, p0_target) (only for valid coords)
        REGv = REG0.copy()
        if np.isfinite(d_spot).any():
            thr = np.quantile(d_spot[np.isfinite(d_spot)], p0_target)
            REGv[:] = "interior"
            idx_ok = np.where(ok_mask)[0]
            REGv[idx_ok[d_spot <= thr]] = "boundary"
        # recount edges with same detection threshold as baseline (log1p CPM > 0)
        df_counts = _recount_edges_with_regions(adata, GRAPH[sid], AXES_USE, REGv, thr_log=0.0)
        # stats & calls vs p0_target
        stats = _stats_calls(df_counts, p0_target)
        for _, r in stats.iterrows():
            calls_for_sid.setdefault(r["axis"], {})[tag] = r["call"]

    # 2) Detection threshold perturbation: CPM≥1 (log1p>0.693), keep baseline rim
    df_counts_thr = _recount_edges_with_regions(adata, GRAPH[sid], AXES_USE, REG0, thr_log=np.log1p(1.0))
    stats_thr = _stats_calls(df_counts_thr, p0_base)
    for _, r in stats_thr.iterrows():
        calls_for_sid.setdefault(r["axis"], {})["thr_cpm≥1"] = r["call"]

    # collect rows
    for axis, d in calls_for_sid.items():
        rows.append({
            "sample_id": sid,
            "axis": axis,
            "base_call": d.get("base_call", "neutral"),
            "rim_-25%": d.get("rim_-25%", "neutral"),
            "rim_+25%": d.get("rim_+25%", "neutral"),
            "thr_cpm≥1": d.get("thr_cpm≥1", "neutral"),
        })

ROBUST_CALLS = pd.DataFrame(rows).sort_values(["axis","sample_id"]).reset_index(drop=True)
ROBUST_CALLS.to_csv(FINAL / "robustness_calls.csv", index=False)

# visible output: show flips (where any variant != base)
ROBUST_CALLS.assign(flipped=((ROBUST_CALLS.filter(["rim_-25%","rim_+25%","thr_cpm≥1"]) != ROBUST_CALLS["base_call"].values[:,None]).any(axis=1))) \
            .sort_values(["axis","sample_id"])


,sample_id,axis,base_call,rim_-25%,rim_+25%,thr_cpm≥1,flipped
0,GSE217414,CXCL12->CXCR4,depleted,neutral,depleted,depleted,True
1,GSE226997,CXCL12->CXCR4,enriched,enriched,enriched,enriched,False
2,GSE274103,CXCL12->CXCR4,enriched,enriched,enriched,enriched,False
3,GSE217414,SPP1->ITG*,enriched,enriched,enriched,enriched,False
4,GSE226997,SPP1->ITG*,enriched,enriched,enriched,enriched,False
5,GSE274103,SPP1->ITG*,enriched,enriched,enriched,enriched,False
6,GSE217414,TGFB1->TGFBR*,enriched,enriched,enriched,enriched,False
7,GSE226997,TGFB1->TGFBR*,enriched,enriched,enriched,enriched,False
8,GSE274103,TGFB1->TGFBR*,neutral,depleted,enriched,neutral,True
9,GSE217414,VEGFA->KDR,enriched,enriched,enriched,enriched,False


In [17]:
# INTERPRETATION — Robustness (rim ±25% and CPM≥1 gating)

# Most axes are stable to reasonable analysis choices:
#   • SPP1→integrins: enriched on all slides under all perturbations → strong, robust rim signal.
#   • VEGFA→KDR: enriched on GSE217414 and GSE274103 under all perturbations; on GSE226997 it is
#     borderline (neutral at baseline/CPM≥1, but enriched when the rim is ±25%), i.e., near the decision boundary.
#   • TGFB1→TGFBR*: enriched on GSE217414 and GSE226997 across settings; GSE274103 flips (neutral → depleted/enriched)
#     indicating low confidence on that slide’s rim assignment for this axis.
#   • CXCL12→CXCR4: context-dependent. Stable enrichment on GSE226997 and GSE274103; on GSE217414 it is depleted at
#     baseline/CPM≥1 but moves to neutral with a thinner rim (−25%). This suggests a weak or patchy rim effect there.
#
# Take-home for IEvI:
#   • The core exclusion signature—VEGFA→KDR and SPP1→integrins concentrating at the tumor rim—is robust.
#   • CXCL12→CXCR4 behaves by niche: interior-skewed in GSE217414, rim-enriched elsewhere.
#   • TGFB1→TGFBR* is generally rim-biased but sensitive on GSE274103; treat that slide’s call as low confidence.
#
# Practical note:
#   • The two flips (TGFB1 on GSE274103; VEGFA on GSE226997) reflect ambiguity in rim geometry or effects near threshold.
#     If available, a curated role_map (tumor mask) for GSE274103 would likely stabilize TGFB1. For reporting, anchor
#     claims on axes/slides without flips and annotate the sensitive cases explicitly.


In [18]:
################### Stage 9 — Distance-Band Profiles (WASR-Style Relocation Analysis)
# why
#   Beyond boundary vs interior, IEvI benefits from a gradient view: how each axis redistributes
#   across increasing distances from the tumor boundary (a WASR-style profile).
#
# how
#   - For each slide, compute per-spot distance to the tumor boundary (same tumor mask logic as Cell 4c).
#   - Define 4 distance bands by within-slide quantiles of distance (scale-free across slides):
#       B1: 0–10% (rim), B2: 10–25%, B3: 25–50%, B4: 50–100% (deep interior/outer stroma).
#   - For each axis × slide, count LR edges whose TARGET spot falls in each band.
#   - Compute share_band = edges_in_band / total_edges (per slide/axis).
#   - Baseline p_band = (# spots in band) / (all spots) on that slide.
#   - Report Δ_band = share_band − p_band, Wilson 95% CI for share_band, and one-sided p-values vs p_band.
#
# what
#   - Writes:
#       final_results/wasr_bands_per_sample.csv
#       final_results/wasr_bands_pooled.csv
#   - Returns a compact pooled table (axes × bands) sorted by |Δ| to preview the main effects.
#
# interpretation
#   - Positive Δ near the rim (B1/B2) = rim-skew (exclusion-like). Positive Δ only in deeper bands = interior-skew.
#   - Because bands are by quantile, p_band is the band width (0.10, 0.15, 0.25, 0.50) per slide.

import math
import numpy as np
import pandas as pd
from pathlib import Path
from scipy import ndimage as ndi
from scipy import sparse as sp

# ---- prerequisites
assert 'PREPROC' in globals() and isinstance(PREPROC, dict) and len(PREPROC)>0, "Run Cells 1–6 first."
assert 'GRAPH'   in globals() and isinstance(GRAPH, dict)   and len(GRAPH)>0, "Run Cell 3 first."
# axes: reuse AXES_DF/AXES or the default
if 'AXES_DF' in globals():
    AXES_USE = AXES_DF.copy()
elif 'AXES' in globals():
    AXES_USE = pd.DataFrame(AXES)
else:
    AXES_USE = pd.DataFrame([
        {"ligand":"VEGFA",  "receptor":"KDR",   "axis":"VEGFA->KDR",      "receptor_is_prefix": False},
        {"ligand":"CXCL12","receptor":"CXCR4", "axis":"CXCL12->CXCR4",   "receptor_is_prefix": False},
        {"ligand":"TGFB1", "receptor":"TGFBR", "axis":"TGFB1->TGFBR*",   "receptor_is_prefix": True},
        {"ligand":"SPP1",  "receptor":"ITG",   "axis":"SPP1->integrins", "receptor_is_prefix": True},
    ])

APP_ROOT = Path(APP_ROOT) if 'APP_ROOT' in globals() else Path.cwd()
FINAL    = Path(FINAL)    if 'FINAL'    in globals() else (APP_ROOT / "final_results")
FINAL.mkdir(parents=True, exist_ok=True)

# ---- helpers
def wilson_ci(k, n, z=1.96):
    if n <= 0: return (np.nan, np.nan)
    p = k / n
    denom = 1 + z*z/n
    center = (p + z*z/(2*n)) / denom
    half = (z/denom) * math.sqrt((p*(1-p)/n) + (z*z/(4*n*n)))
    return (max(0.0, center - half), min(1.0, center + half))

def pvals_vs_p0(k, n, p0):
    if n <= 0 or not (0 < p0 < 1): return (np.nan, np.nan)
    mean = n * p0
    var  = n * p0 * (1 - p0)
    if var <= 0: return (np.nan, np.nan)
    z = (k - mean) / math.sqrt(var)
    p_enrich  = 0.5 * math.erfc(z / math.sqrt(2))
    p_deplete = 1.0 - p_enrich
    return (max(0.0, min(1.0, p_enrich)), max(0.0, min(1.0, p_deplete)))

def _to_grid(xy, max_w=768):
    x = xy[:,0].astype(float); y = xy[:,1].astype(float)
    x -= x.min(); y -= y.min()
    w = max(x.max(), 1.0); h = max(y.max(), 1.0)
    scale = max_w / max(w, h)
    gw = int(np.ceil(w * scale)) + 1
    gh = int(np.ceil(h * scale)) + 1
    gx = np.clip((x * scale).round().astype(int), 0, gw-1)
    gy = np.clip((y * scale).round().astype(int), 0, gh-1)
    return gx, gy, gw, gh

def _tumor_mask_from_points(gx_t, gy_t, gw, gh, sigma=3.5):
    img = np.zeros((gh, gw), dtype=np.float32)
    if gx_t.size:
        np.add.at(img, (gy_t, gx_t), 1.0)
    img = ndi.gaussian_filter(img, sigma)
    if (img > 0).any():
        thr = np.quantile(img[img > 0], 0.90)  # keep bright core; matches Cell 4c default (1-THRESH_Q with 0.10)
    else:
        thr = 0.0
    mask = img >= thr
    # cleanup
    struc = ndi.generate_binary_structure(2, 1)
    mask = ndi.binary_closing(mask, structure=struc, iterations=1)
    mask = ndi.binary_fill_holes(mask)
    return mask

def _distance_to_tumor_boundary(adata):
    XY = np.asarray(adata.obsm["spatial"], dtype=float)
    ok = np.isfinite(XY).all(axis=1)
    XY_ok = XY[ok]
    role = adata.obs.get("role", pd.Series(["stroma"]*adata.n_obs)).astype(str).values
    tumor_ok = (role[ok] == "tumor")
    gx, gy, gw, gh = _to_grid(XY_ok, max_w=768)
    mask = _tumor_mask_from_points(gx[tumor_ok], gy[tumor_ok], gw, gh, sigma=3.5)
    if not mask.any():
        d = np.full(XY_ok.shape[0], np.inf, dtype=float)
    else:
        dist_in  = ndi.distance_transform_edt(mask)
        dist_out = ndi.distance_transform_edt(~mask)
        d = np.where(mask, dist_in, dist_out).astype(float)
    # write back to obs as dist_to_tumor_boundary (np.inf becomes NaN)
    d_full = np.full(adata.n_obs, np.nan, dtype=float)
    d_full[np.where(ok)[0]] = d[gy, gx]
    adata.obs["dist_to_tumor_boundary"] = d_full
    return d_full

def _present_any(adata, gene_list):
    if not gene_list: 
        return np.zeros(adata.n_obs, dtype=bool)
    name_to_idx = {str(g): i for i, g in enumerate(adata.var_names)}
    cols = [name_to_idx[g] for g in gene_list if g in name_to_idx]
    if not cols:
        return np.zeros(adata.n_obs, dtype=bool)
    X = adata.X
    if sp.issparse(X):
        sub = X.tocsc()[:, cols]
        return np.asarray(sub.sum(axis=1) > 0).ravel()
    else:
        sub = X[:, cols]
        return (sub.sum(axis=1) > 0)

def _present_prefix(adata, prefix):
    cols = [i for i, g in enumerate(adata.var_names) if str(g).startswith(prefix)]
    if not cols:
        return np.zeros(adata.n_obs, dtype=bool)
    X = adata.X
    if sp.issparse(X):
        sub = X.tocsc()[:, cols]
        return np.asarray(sub.sum(axis=1) > 0).ravel()
    else:
        sub = X[:, cols]
        return (sub.sum(axis=1) > 0)

# ---- define bands (quantiles within each slide)
BANDS = [
    (0.00, 0.10, "B1: 0–10% (rim)"),
    (0.10, 0.25, "B2: 10–25%"),
    (0.25, 0.50, "B3: 25–50%"),
    (0.50, 1.00, "B4: 50–100%"),
]

# ---- compute distances + bands, then edge counts per band
per_rows = []   # per-sample × axis × band
for sid, adata in PREPROC.items():
    if ("spatial" not in adata.obsm) or (sid not in GRAPH):
        continue

    # distances
    d = adata.obs.get("dist_to_tumor_boundary", None)
    if d is None or pd.isna(d).all():
        d = _distance_to_tumor_boundary(adata)

    df = pd.DataFrame({"d": d})
    df = df.dropna()
    if df.empty:
        continue

    # band thresholds by quantiles
    qs = [0.0] + sorted({q for a,b,_ in BANDS for q in (a,b)}) + [1.0]
    qs = [q for q in qs if 0.0 <= q <= 1.0]
    # unique + sorted
    qs = np.unique(np.array(qs))
    qvals = np.quantile(df["d"].values, qs)

    # map each spot to a band label
    labels = np.full(adata.n_obs, "B4: 50–100%", dtype=object)  # default deepest band
    for (qa, qb, lab) in BANDS:
        lo = np.quantile(df["d"].values, qa)
        hi = np.quantile(df["d"].values, qb)
        sel = (adata.obs["dist_to_tumor_boundary"] >= lo) & (adata.obs["dist_to_tumor_boundary"] < hi)
        labels[sel.fillna(False).values] = lab
    adata.obs["dist_band"] = labels

    # baseline band occupancy (p_band)
    band_counts = pd.Series(adata.obs["dist_band"], dtype="object").value_counts().to_dict()
    n_spots = int(getattr(adata, "n_obs", 0))
    p_band = {lab: (band_counts.get(lab, 0) / n_spots if n_spots > 0 else np.nan) for _,_,lab in BANDS}

    # edge counting per axis × band (TARGET band)
    g = GRAPH[sid]
    src = g["edge_src"].astype(np.int32)
    dst = g["edge_dst"].astype(np.int32)
    REG_BAND = adata.obs["dist_band"].astype(str).values

    cache = {}
    for _, ax in AXES_USE.iterrows():
        lig = str(ax["ligand"]); rec = str(ax["receptor"]); pref = bool(ax["receptor_is_prefix"])
        if ("L", lig) not in cache:
            cache[("L", lig)] = _present_any(adata, [lig])
        lig_src = cache[("L", lig)]
        key = ("R*", rec) if pref else ("R", rec)
        if key not in cache:
            cache[key] = _present_prefix(adata, rec) if pref else _present_any(adata, [rec])
        rec_tgt = cache[key]

        ok = lig_src[src] & rec_tgt[dst]
        tot = int(ok.sum())
        # counts per band
        for _,_,lab in BANDS:
            if tot == 0:
                b = 0
            else:
                b = int((ok & (REG_BAND[dst] == lab)).sum())
            share = (b / tot) if tot > 0 else np.nan
            lo, hi = wilson_ci(b, tot) if tot > 0 else (np.nan, np.nan)
            p0 = p_band.get(lab, np.nan)
            pe, pdp = pvals_vs_p0(b, tot, p0) if (tot > 0 and not np.isnan(p0)) else (np.nan, np.nan)
            per_rows.append({
                "sample_id": sid,
                "axis": f"{lig}->{rec}" + ("*" if pref else ""),
                "band": lab,
                "edges_in_band": b,
                "edges_total": tot,
                "share_band": share,
                "p_band": p0,
                "delta": (share - p0) if (not np.isnan(share) and not np.isnan(p0)) else np.nan,
                "ci_lo": lo, "ci_hi": hi,
                "p_enrich": pe, "p_deplete": pdp,
            })

# assemble per-sample table
WASR_PER = pd.DataFrame(per_rows)

# pooled across samples per axis × band (edges-weighted)
def _weighted_p_band(g):
    # weight p_band by number of spots? Simpler: average p_band weighted by edges_total, to match share weighting
    w = g["edges_total"].sum()
    if w == 0: return np.nan
    return (g["p_band"] * g["edges_total"]).sum() / w

pooled = (WASR_PER
          .groupby(["axis","band"], as_index=False)
          .agg(edges_in_band=("edges_in_band","sum"),
               edges_total=("edges_total","sum")))
pooled["share_band"] = pooled["edges_in_band"] / pooled["edges_total"].replace({0:np.nan})
# pooled baseline
p0w = (WASR_PER
       .groupby(["axis","band"])
       .apply(_weighted_p_band)
       .reset_index(name="p_band"))
WASR_POOLED = pooled.merge(p0w, on=["axis","band"], how="left")
# CI + p-values for pooled
cis = WASR_POOLED.apply(lambda s: wilson_ci(int(s["edges_in_band"]), int(s["edges_total"])) if s["edges_total"]>0 else (np.nan,np.nan), axis=1)
WASR_POOLED["ci_lo"] = [c[0] for c in cis]
WASR_POOLED["ci_hi"] = [c[1] for c in cis]
pvs = WASR_POOLED.apply(lambda s: pvals_vs_p0(int(s["edges_in_band"]), int(s["edges_total"]), float(s["p_band"]))
                        if (s["edges_total"]>0 and not np.isnan(s["p_band"])) else (np.nan,np.nan), axis=1)
WASR_POOLED["p_enrich"]  = [p[0] for p in pvs]
WASR_POOLED["p_deplete"] = [p[1] for p in pvs]
WASR_POOLED["delta"] = WASR_POOLED["share_band"] - WASR_POOLED["p_band"]

# save
WASR_PER.to_csv(FINAL / "wasr_bands_per_sample.csv", index=False)
WASR_POOLED.sort_values(["axis","band"]).to_csv(FINAL / "wasr_bands_pooled.csv", index=False)

# visible preview: biggest pooled shifts (|Δ|), axis × band
WASR_POOLED.assign(abs_delta=lambda d: d["delta"].abs()) \
           .sort_values(["axis","abs_delta"], ascending=[True, False]) \
           .loc[:, ["axis","band","edges_in_band","edges_total","share_band","p_band","delta","ci_lo","ci_hi"]] \
           .reset_index(drop=True)


,axis,band,edges_in_band,edges_total,share_band,p_band,delta,ci_lo,ci_hi
0,CXCL12->CXCR4,B4: 50–100%,2318,5300,0.437358,0.514821,-0.077463,0.424053,0.450754
1,CXCL12->CXCR4,B3: 25–50%,2095,5300,0.395283,0.354004,0.041279,0.382201,0.408517
2,CXCL12->CXCR4,B2: 10–25%,445,5300,0.083962,0.054779,0.029183,0.076794,0.091733
3,CXCL12->CXCR4,B1: 0–10% (rim),442,5300,0.083396,0.076396,0.007001,0.076251,0.091145
4,SPP1->ITG*,B4: 50–100%,21073,44719,0.471231,0.521054,-0.049822,0.466608,0.475860
5,SPP1->ITG*,B1: 0–10% (rim),4477,44719,0.100114,0.083048,0.017066,0.097366,0.102930
6,SPP1->ITG*,B3: 25–50%,14538,44719,0.325097,0.308174,0.016923,0.320770,0.329453
7,SPP1->ITG*,B2: 10–25%,4631,44719,0.103558,0.087725,0.015833,0.100768,0.106416
8,TGFB1->TGFBR*,B4: 50–100%,16246,33992,0.477936,0.529660,-0.051724,0.472629,0.483248
9,TGFB1->TGFBR*,B3: 25–50%,11288,33992,0.332078,0.296491,0.035587,0.327091,0.337104


In [ ]:
# INTERPRETATION — Distance-band (WASR-style) profiles pooled across slides
# Bands are quantiles of distance to the tumor boundary: B1 (0–10%, rim), B2 (10–25%), B3 (25–50%), B4 (50–100%, deep interior).
# We compare each axis’s edge share in a band to that band’s baseline occupancy (Δ = share_band − p_band).
#
# • SPP1→integrins (ECM/adhesion): clear near-rim skew. B1 (+0.017), B2 (+0.016), B3 (+0.017) are all enriched,
#   while B4 (deep interior) is depleted (−0.050). This reads as a stromal/adhesive “shell” spanning the rim and
#   immediate interior—consistent with barrier-like remodeling at the tumor perimeter.
#
# • VEGFA→KDR (angiogenesis): enrichment peaks in the inner bands rather than the very edge: B3 (+0.032) and B2 (+0.011)
#   are positive; B1 (rim) is ~baseline (+0.001), and B4 is depleted (−0.045). Angiogenic interfaces concentrate just
#   inside the rim, matching the idea of a peritumoral vascular zone rather than an ultra-thin edge effect.
#
# • TGFB1→TGFBR* (TGFβ signaling): similar to SPP1 but slightly shifted inward. B3 (+0.036) and B2 (+0.019) are enriched,
#   rim B1 is ~neutral (−0.002), and deep interior B4 is depleted (−0.052). This suggests broad stromal activation that
#   peaks in the peritumoral zone, not strictly at the boundary pixels.
#
# • CXCL12→CXCR4 (chemotaxis): mid-zone preference. B3 (+0.041) and B2 (+0.029) are enriched; B1 (rim) is only mildly
#   positive (+0.007), and deep interior B4 is depleted (−0.077). Pooled across slides, CXCL12 signaling favors the bands
#   just inside the rim rather than the deepest interior or the razor-thin edge—consistent with chemotactic corridors
#   near the tumor perimeter.
#
# Net readout: the gradient view reinforces IEvI—matrix/adhesion (SPP1→integrins) and angiogenic (VEGFA→KDR) interfaces
# concentrate from the rim into the immediate interior, while TGFβ peaks slightly further in, and CXCL12 skews to mid-bands.
# All axes are depleted in the deep interior (B4), supporting a perimeter-centric signaling niche rather than uniform spread.
